## Capstone Project - Wifi Fingerprinting  
https://archive.ics.uci.edu/ml/datasets/UJIIndoorLoc  

Goal: Read in tons of data, use to predict location  
(secondary: hopefully this will be a good learning opportunity to delve into numpy and pandas idiosyncracies, as well as ways to manipulate all rows/columns at once.  The R version of this project delved into matrix manipulation)  
  
Attributes:  
001-520: WAP signal strengths.  Negative integer values from -104 to 0 and +100. Positive value 100 used if WAP was not detected.  
521-529:  
    521 (Longitude): Longitude. Negative real values from -7695.9387549299299000 to -7299.786516730871000  
    522 (Latitude): Latitude. Positive real values from 4864745.7450159714 to 4865017.3646842018.  
    523 (Floor): Altitude in floors inside the building. Integer values from 0 to 4.  
    524 (BuildingID): ID to identify the building. Measures were taken in three different buildings. Categorical integer values from 0 to 2.  
    525 (SpaceID): Internal ID number to identify the Space (office, corridor, classroom) where the capture was taken. Categorical integer values.  
    526 (RelativePosition): Relative position with respect to the Space (1 - Inside, 2 - Outside in Front of the door). Categorical integer values.  
    527 (UserID): User identifier (see below). Categorical integer values.  
    528 (PhoneID): Android device identifier (see below). Categorical integer values.  
    529 (Timestamp): UNIX Time when the capture was taken. Integer value.   
      
When I did this in R I came up with some unique methods that ended up not working very well.  With Python, I want to try similar methods, but a different top down approach - to start, I want to try predicting building, then floor, then space, then position instead of predicting all 4 directly.  I am also mulling predicting Longitude/Latitude instead, and then using the results for that to predict the unique space.

One good thing about this nested way of predicting is that we can try different methods at the different levels.  We might have 2 or 3 methods to predict building, and then move forward with the best.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#added to try to solve SettingWithCopy warning
#https://www.dataquest.io/blog/settingwithcopywarning/
pd.set_option('mode.chained_assignment', 'raise')

In [2]:
rawdata = pd.read_csv('Z:/C5T4 - Capstone/UJIndoorLoc/trainingData.csv', header =0)

In [3]:
#what do we have?
rawdata.info()
#note row and column count for future streamlining to come

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19937 entries, 0 to 19936
Columns: 529 entries, WAP001 to TIMESTAMP
dtypes: float64(2), int64(527)
memory usage: 80.5 MB


In [4]:
#what's it look like?
rawdata.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


In [5]:
#Effective coding is all about mooching off past programs to save time
#I know from past exploration of the dataset that there are some WAP variables that have all 100s (x54);
#i.e., no readings and thus no predictive value.  We can remove them right now to make things a bit smaller

#note: min=100 for WAP variables means no readings, but it is possible this criteria could exclude other non-WAP variables
#I happen to know that the other 10 variables would not be affected by this criteria, but this could be confirmed with summary statistics

#failed attempts to limit using a single line 
#cleandata = rawdata[c for c in rawdata.columns if rawdata.c.min()<>100]
#cleandata = np.where[c for c in rawdata.columns if rawdata.c.min()<>100]
#cleandata = rawdata[rawdata.min != 100]
#cleandata = rawdata[rawdata.index.min() != 100]
#so time to break it down

#returns 2 columns, first is attribute name and second is min of each column
#>rawdata.min(axis=0)
#WAP001             -9.700000e+01
#WAP002             -9.000000e+01
#WAP003              1.000000e+02

#returns 2 columns, first is attribute name and second is boolean of whether min of each column does not equal 100 or not
mask=rawdata.min(axis=0) !=100
#>print(mask)
#WAP001               True
#WAP002               True
#WAP003              False

#removes non-true
mask2 = mask[mask == True]
#WAP001              True
#WAP002              True
#WAP005              True

#>mask2.index
#'WAP001', 'WAP002', 'WAP005',

#calls mask2 by index, which are the attribute names
#hopefully there is a more elegant solution out there, but this is the one I found that works
tempdata=rawdata[mask2.index].copy()

#to circle back, to make it a two liner, the first being 'mask', see below
#cleandata=rawdata[mask[mask == True].index]

#this one took me 3 lines in R using apply to find min, then subset to remove the == 100, then a third to apply the variable list

In [6]:
tempdata.shape
#this one should be
#(19937, 474)

(19937, 474)

In [7]:
tempdata.info()
#note number of columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19937 entries, 0 to 19936
Columns: 474 entries, WAP001 to TIMESTAMP
dtypes: float64(2), int64(472)
memory usage: 72.1 MB


In [8]:
#Similarly, there are some observations (x76) with no WAP signal strengths.  These represent a multitude of places, 
#thus can't be effectively lumped into one, so we'll remove them

#we want observations where all of the WAP variables are 100, but the non-WAP attributes can be anything
#there are 9 non-WAP attributes
#there is probably a better way, but I think I will do it like this:
#1) munge a new variable, whether the WAPs are all 100
#2) filter on that variable
#3) remove the munged variable
#if I can get that to work, then maybe I can make it into a more elegant or more succinct one liner

#but first, lets see if we can make an array of 100s and just see if we can filter it out
#temp = cleandata.iloc[:,0:465].values == np.full(465, 100)
#unfortunately this gives us a True/False for each of 474 elements (then x each row)

#this one seems to get what we want, a single value for each row
all100mask = (tempdata.iloc[:,0:465].values == np.full(465, 100)).all(axis=1)
print(all100mask.shape)
#(19937,)
print(all100mask)
#array([False, False, False, ..., False, False, False])
print(all100mask[all100mask==True].shape)
#(76,)

#so now we just need to filter by the boolean
#unlike the previous mask, we do not have column names to index by
print(tempdata[all100mask==True].shape)
#(76, 474)
#we want the rest
tempdata2 = tempdata[all100mask==False].copy()
#clear tempdata to save memory
tempdata = None

#This one took me 1 line in R, using subset with a nested apply

(19937,)
[False False False ... False False False]
(76,)
(76, 474)


In [9]:
#confirm we haven't screwed the pooch
tempdata2.shape
#excellent, 19937 - 76 = 19861 rows

(19861, 474)

In [10]:
#debug to see if the removal of the non-changing rows worked,
#row ID 4 should be missing in this
tempdata2.head(6)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,WAP519,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,-97,100,100,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
5,100,100,100,100,100,100,100,100,100,100,...,100,-7533.8962,4.864939e+06,2,1,105,2,2,23,1371713841
6,100,100,100,100,100,100,100,100,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371713883


In [11]:
cleandata=tempdata2.copy()
#clear tempdata2
tempdata2 = None
#this redundant step was due to earlier testing; right now easier to leave it in then change variable names across steps

#My general approach is to munge some variables that might have meaning, based on my understanding of the data
#For small datasets, I keep the originals in too, as to not lose data
#For this one, there are so many variables that I ended up removing all of the WAP variables in R before
#doing the learning model and just sticking with the munged ones.  I think I will do that again here
#The risk is that I will lose possible data by removing those variables.  If my assumptions are wrong, or
#missing a large predictive factor, then my predictions may not be as accurate

#So, starting off with predicting building
#When doing the prediction, we will be looking at WAP readings and trying to guess which building
#Some WAPs may only be associated with one building, some may be associated with more
#Here is how I imagine a WAP based fingerprint at the building level working
#For a given WAPX, we can see how many observations are for building 0, then for building 1, then for building 2
#Then we can store these counts
#Then, over all of the WAPs for an observation, we can add the building 0 counts, building 1 counts, and building 2 counts
#From there, some pairwise variables.  An absolute count for building 0 will not necessarily help us,
#until we also know how many building 1 counts and building 2 counts we have
#So, we can munge ratio variables to compare one to another
#As another possibility, we could normalize the counts by total counts
#Just a straight count could bias the data towards places that are visited more often than others.  That said,
#that may not be a bad thing from a predictive point of view, from a bayesian standpoint
#One thing I noticed in my original run through and tried to correct via weighting is that some WAPs are only observed
#occasionally from a given.  Counting the observations will incorporate this weighting implicitly

#If we are going to get building counts for each WAP, we'll need to start with a unique list of WAPs
WAPs = np.array(cleandata.iloc[:,0:465].columns.values)
#didn't end up using this, see below

WAPs.shape
(465,)

#append the three building counts
WAPs2 = np.concatenate((WAPs, np.empty([465,3])), axis=0)

#doesn't work
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-13-8eeec22417d0> in <module>()
      1 #append the three building counts
----> 2 WAPs2 = np.concatenate((WAPs, np.empty([465,3])), axis=0)

ValueError: all the input arrays must have same number of dimensions



np.empty([465,3]).shape
(465, 3)

WAPs.shape
(465,)

#it's not working (yet)
#maybe assigning the names as an array entry is not what we need anyway; not sure exactly how they will interface
#so I guess will start trying to do the group by to get the values, and then see if we want to add the names on as an attribute,
#or add them as names
grouped = cleandata.groupby('BUILDINGID')
print(grouped["WAP001"].count)

<bound method SeriesGroupBy.count of <pandas.core.groupby.SeriesGroupBy object at 0x00000000092E4908>>

cleandata.iloc[:,0:465].apply(lambda x: 1*(x<100.0), axis=1).groupby('BUILDINGID').sum(axis=0)

#doesn't work because group by field isn't in data
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-17-b08f5c81b8ac> in <module>()
----> 1 cleandata.iloc[:,0:465].apply(lambda x: 1*(x<100.0), axis=1).groupby('BUILDINGID').sum(axis=0)

~\Anaconda3\lib\site-packages\pandas\core\generic.py in groupby(self, by, axis, level, as_index, sort, group_keys, squeeze, **kwargs)
   5160         return groupby(self, by=by, axis=axis, level=level, as_index=as_index,
   5161                        sort=sort, group_keys=group_keys, squeeze=squeeze,
-> 5162                        **kwargs)
   5163 
   5164     def asfreq(self, freq, method=None, how=None, normalize=False,

~\Anaconda3\lib\site-packages\pandas\core\groupby.py in groupby(obj, by, **kwds)
   1846         raise TypeError('invalid type: %s' % type(obj))
   1847 
-> 1848     return klass(obj, by, **kwds)
   1849 
   1850 

~\Anaconda3\lib\site-packages\pandas\core\groupby.py in __init__(self, obj, keys, axis, level, grouper, exclusions, selection, as_index, sort, group_keys, squeeze, **kwargs)
    514                                                     level=level,
    515                                                     sort=sort,
--> 516                                                     mutated=self.mutated)
    517 
    518         self.obj = obj

~\Anaconda3\lib\site-packages\pandas\core\groupby.py in _get_grouper(obj, key, axis, level, sort, mutated, validate)
   2932                 in_axis, name, level, gpr = False, None, gpr, None
   2933             else:
-> 2934                 raise KeyError(gpr)
   2935         elif isinstance(gpr, Grouper) and gpr.key is not None:
   2936             # Add key to exclusions

KeyError: 'BUILDINGID'

In [12]:
#I can't seem to get a function to apply to a subset of attributes but then retain the other attributes for grouping purposes
#so will have to break it up into multiple steps

WAPdata = pd.concat((cleandata.iloc[:,0:465].apply(lambda x: 1*(x<100.0)),cleandata.iloc[:,465:474]), axis = 1).copy()
WAPdata.head(3)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,WAP519,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,0,0,0,0,0,0,0,0,0,0,...,0,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,0,0,0,0,0,0,0,0,0,0,...,0,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,0,0,0,0,0,1,0,0,0,0,...,0,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095


In [13]:
WAPdata.shape

(19861, 474)

In [14]:
WAPdata.groupby('BUILDINGID').sum(axis=0)
#i can't for the life of me get python to do a noncontiguous iloc to include building ID but not the other non-WAPs
#iloc only takes in 1 range
#even setting it as a list, list itself only takes in 1 range
#dumb dumb dumb dumb dumb
#we'll just have to ignore those others or drop them or something
#...of course, drop probably wouldn't work either, because there are two noncontiguous ranges to drop

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,WAP518,WAP519,LONGITUDE,LATITUDE,FLOOR,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
BUILDINGID,,,,,,,,,,,,,,,,,,,,,
0,18,19,0,0,578,121,595,0,0,0,...,0,3,-4.009111e+07,2.553130e+10,8415,881657,10467,30358,70961,7192845651045
1,0,0,0,280,0,556,0,87,112,104,...,0,0,-3.863287e+07,2.509794e+10,7009,688632,8694,53195,65655,7076691401067
2,0,0,40,28,0,0,0,0,2844,2879,...,22,0,-6.952580e+07,4.599197e+10,17848,1379543,17253,96497,122602,12968239674060


In [15]:
buildingdata = WAPdata.iloc[:,0:470].drop(['LONGITUDE','LATITUDE','FLOOR','SPACEID'], axis=1).groupby('BUILDINGID').sum(axis=0).copy()
#managed to get it to work.  had to browbeat it manually though, really hard
#drop a range?  nope
#drop a list of numbers?  nope.  rows work i guess, but not columns

In [16]:
#now need similar sublevels for future
floordata = WAPdata.iloc[:,0:470].drop(['LONGITUDE','LATITUDE','SPACEID'], axis=1).groupby(['BUILDINGID','FLOOR']).sum(axis=0).copy()
#roomdata = WAPdata.iloc[:,0:470].drop(['LONGITUDE','LATITUDE'], axis=1).groupby(['BUILDINGID','FLOOR','SPACEID']).sum(axis=0).copy()
#relposdata = WAPdata.iloc[:,0:471].drop(['LONGITUDE','LATITUDE'], axis=1).groupby(['BUILDINGID','FLOOR','SPACEID','RELATIVEPOSITION']).sum(axis=0).copy()

#!!these end up not being used, could not figure out how to call the multiple level hierarchy
#see testing later, which is why floordata is left in
#similar made later, except with 'as_index = False' added so grouped by labels repeat
#and thus can be called like a normal dataframe, filtered

In [17]:
#alright, so now we have our building grouping with counts of non-100 observations
#time to munge the variables

#here is what we want to do for each observation:
#add up the building 0, 1, and 2 counts for each WAP variable
#munge some ratio and difference variables

#let's define some functions
#first, for a particular observation, we want the columns with non-100 values
#this will get used for each hierarchy level, not just building
def applWAPs(observation):    #applicable WAPs
    mask3=observation !=100
    mask4 = mask3[mask3 == True]
    #calls mask4 by index, which are the attribute names
    return mask4.index


In [18]:
#test2 calling the building data array using the function as the columns
buildingdata[applWAPs(cleandata.iloc[0,0:465])]

,WAP090,WAP091,WAP103,WAP104,WAP105,WAP106,WAP125,WAP126,WAP166,WAP167,WAP168,WAP169,WAP171,WAP172,WAP173,WAP191,WAP192
BUILDINGID,,,,,,,,,,,,,,,,,
0,0,0,6,5,0,0,0,0,55,57,3,6,1,163,155,9,9
1,1006,1012,935,1022,926,939,789,798,1085,1106,654,666,880,1029,1043,569,582
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
#test3 coming up with the by building sums
buildingdata[applWAPs(cleandata.iloc[0,0:465])].sum(axis=1)

BUILDINGID
0      469
1    15041
2        0
dtype: int64

In [20]:
#final individual test, assigning the data
buildingdata[applWAPs(cleandata.iloc[0,0:465])].sum(axis=1)[0]

469

In [21]:
#some test data for doing a group of them
print(buildingdata[applWAPs(cleandata.iloc[0,0:465])].sum(axis=1))
print(buildingdata[applWAPs(cleandata.iloc[1,0:465])].sum(axis=1))
print(buildingdata[applWAPs(cleandata.iloc[2,0:465])].sum(axis=1))
print(buildingdata[applWAPs(cleandata.iloc[3,0:465])].sum(axis=1))

BUILDINGID
0      469
1    15041
2        0
dtype: int64
BUILDINGID
0      367
1    13582
2        0
dtype: int64
BUILDINGID
0     1100
1    13400
2     2599
dtype: int64
BUILDINGID
0     2040
1    13486
2        0
dtype: int64


apply(buildingdata[cleandata.iloc[0:4,0:465].applWAPs],axis=1).sum(axis=1)
#this isn't going to work as is.  Need to make another function that can be used in the apply

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-64-ba036472984f> in <module>()
----> 1 apply(buildingdata[cleandata.iloc[0:4,0:465].applWAPs],axis=1).sum(axis=1)

NameError: name 'apply' is not defined

In [22]:
#function to return the buildingdata[applWAPs(cleandata.iloc[0,0:465])].sum(axis=1) type data
#and then maybe generalize to send back all 3 variables at once to do a single call?
#(efficiency, so that the same aggregation is not happening 3 times)

def buildingsums (observation):
    bsums = buildingdata[applWAPs(observation)].sum(axis=1)
    return bsums[0], bsums[1], bsums[2]

In [23]:
#test
buildingsums(cleandata.iloc[0,0:465])
#fantabular

(469, 15041, 0)

In [24]:
#apply method to get all
cleandata.iloc[0:4,0:465].apply(buildingsums,axis=1)

0        (469, 15041, 0)
1        (367, 13582, 0)
2    (1100, 13400, 2599)
3       (2040, 13486, 0)
dtype: object

In [25]:
#to add to an array
temparray = cleandata[0:4].copy()
temparray2 = pd.concat((temparray, temparray.iloc[:,0:465].apply(buildingsums,axis=1)), axis=1).copy()
print(temparray2)

#can we call one of them?
#nope, can't figure out how to call the nested tuple by name or numerical index
#temparray.iloc[0,475][1]

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   

   WAP012         ...           LONGITUDE      LATITUDE  FLOOR  BUILDINGID  \
0     100         ...          -7541.2643  4.864921e+06      2           1   
1     100         ...          -7536.6212  4.864934e+06      2           1   
2     100         ...          -7519.1524  4.864950e+06      2           1   
3     100         ...          -7524.5704  4.864934e+06      2           1   

   SPACEID  RELATIVEPOSITION  USERID  PHONEID   TIMESTAMP                    0  
0      106                 2       2       23  1371713733      (469, 15041, 0)  
1      106                 2       2       23  1371713691    

#let's see if we can do it other ways
temparray = cleandata[0:4].copy()
temparray[('bldg0sum','bldg1sum','bldg2sum')] =temparray.iloc[:,0:465].apply(buildingsums,axis=1).copy()
print(temparray)
#same result

#can we call one of them?
#temparray['bldg0sum']
#nope

In [26]:
#lets try a tweaked function

def buildingsums2 (observation):
    bsums2 = buildingdata[applWAPs(observation)].sum(axis=1).copy()
    return {'bldg0sum':bsums2[0], 'bldg1sum':bsums2[1], 'bldg2sum':bsums2[2]}

In [27]:
#to add to an array
temparray = cleandata.iloc[0:4,:].copy()
temparray2 = pd.concat((temparray, temparray.iloc[:,0:465].apply(buildingsums2,axis=1)), axis=1).copy()
print(temparray2)
#nope, still a tuple

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   

   WAP012                        ...                          LONGITUDE  \
0     100                        ...                         -7541.2643   
1     100                        ...                         -7536.6212   
2     100                        ...                         -7519.1524   
3     100                        ...                         -7524.5704   

       LATITUDE  FLOOR  BUILDINGID  SPACEID  RELATIVEPOSITION  USERID  \
0  4.864921e+06      2           1      106                 2       2   
1  4.864934e+06      2           1      106                 2       2   
2  4.864950e+06    

#ok, lets try to unpack then
#let's see if we can do it other ways
temparray = cleandata[0:4]
temparray['temp'] =temparray.iloc[:,0:465].apply(buildingsums,axis=1)
print(temparray)

print(temparray['temp']) #gets whole tuple field
print(temparray['temp'][0])#gets first observation, all 3 columns of tuple
#print(temparray['temp'][0,0])  fails
print(temparray['temp'][0][0])#gets first observation, first column of tuple
#try to get all of first column of tuple
#print(temparray['temp'][][0]) #fails
print(temparray['temp'][:][0]) #fails, returns whole first observation tuple
print(temparray['temp'][[0]]) #fails, returns whole first observation tuple, with '0' tuple label
print(temparray['temp'].loc[0][0]) #gets first observation, first column of tuple
#print(temparray['temp'].loc[][0]) #fails
print(temparray['temp'].loc[:][0]) #fails, returns whole first observation tuple
#print(temparray['temp'].loc[:,0]) #fails, too many indexers

#looking up how to unpack tuples
print(*temparray['temp']) #returns all in a big list
print(*temparray['temp'][0]) #returns observation 0 in a list?
#print(*temparray['temp'][0].shape)  #fails, tuple has no shape

#more research
#temparray[('bldg0sum','bldg1sum','bldg2sum')] = zip(*temparray['temp'])
#print(temparray)
#no dice, lets try to get a single zip example to work
#zip(*temparray['temp'][0])  failed, apparently tuples don't iterate.  Boy was I steered wrong

#and yet more searching
temparray['temp'].apply(pd.Series, index=["bldg0sum","bldg1sum", "bldg2sum"])
#omfg, that may have actually worked

#so...to unpack
#temparray[('bldg0sum','bldg1sum','bldg2sum')]=temparray['temp'].apply(pd.Series,index=["bldg0sum","bldg1sum", "bldg2sum"])
#whew.  For a second I thought it was actually going to work.  Balance has been restored to 100% not work
temparray2 = pd.concat((temparray,temparray['temp'].apply(pd.Series,index=["bldg0sum","bldg1sum", "bldg2sum"])),axis=1)
print(temparray2)
#holy moly, it worked
#saving
temparray2.drop('temp', axis=1, inplace=True)
print(temparray2)

In [28]:
#so, let's see if we can do this on the whole enchilada
#what could go wrong?
cleandataonepointfive = pd.concat((cleandata, cleandata.iloc[:,0:465].apply(buildingsums,axis=1).rename('temp')),axis=1).copy()
cleandataonepointfive.head(6)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,temp
0,100,100,100,100,100,100,100,100,100,100,...,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733,"(469, 15041, 0)"
1,100,100,100,100,100,100,100,100,100,100,...,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691,"(367, 13582, 0)"
2,100,100,100,100,100,-97,100,100,100,100,...,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095,"(1100, 13400, 2599)"
3,100,100,100,100,100,100,100,100,100,100,...,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807,"(2040, 13486, 0)"
5,100,100,100,100,100,100,100,100,100,100,...,-7533.8962,4.864939e+06,2,1,105,2,2,23,1371713841,"(1623, 11490, 0)"
6,100,100,100,100,100,100,100,100,100,100,...,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371713883,"(1057, 15181, 2599)"


In [29]:
cleandata2=pd.concat((cleandataonepointfive, cleandataonepointfive.loc[:,'temp'].apply(pd.Series,index=["bldg0sum","bldg1sum", "bldg2sum"])), axis=1).copy()
print(cleandata2.head(6))

#clear variable
cleandataonepointfive = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012    ...     BUILDINGID  SPACEID  RELATIVEPOSITION  USERID  PHONEID  \
0     100    ...              1      106                 2       2       23   
1     100    ...              1      106                 2       2       23   
2     100    ...              1      103                 2       2       23   
3     100    ...              1      102                 2       2       23   
5     100    ...              1      105                 2       

In [30]:
cleandata2.drop('temp', axis=1, inplace=True)
cleandata2.head(6)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,bldg0sum,bldg1sum,bldg2sum
0,100,100,100,100,100,100,100,100,100,100,...,2,1,106,2,2,23,1371713733,469,15041,0
1,100,100,100,100,100,100,100,100,100,100,...,2,1,106,2,2,23,1371713691,367,13582,0
2,100,100,100,100,100,-97,100,100,100,100,...,2,1,103,2,2,23,1371714095,1100,13400,2599
3,100,100,100,100,100,100,100,100,100,100,...,2,1,102,2,2,23,1371713807,2040,13486,0
5,100,100,100,100,100,100,100,100,100,100,...,2,1,105,2,2,23,1371713841,1623,11490,0
6,100,100,100,100,100,100,100,100,100,100,...,2,1,103,2,2,23,1371713883,1057,15181,2599


In [31]:
#now, to make the other munged variables for the first learning model to use
#let's start with a function to 'guess' which building is the winner - the max
def bldgguess(observation):
    bldgnum = -1
    if observation['bldg0sum']>observation['bldg1sum']:
        if observation['bldg0sum']>observation['bldg2sum']:
            bldgnum = 0
        else: bldgnum = 2
    elif observation['bldg1sum']>observation['bldg2sum']:
        bldgnum = 1
    else: bldgnum = 2
    #note, ties will default to 1 for 0 & 1, 2 for 0 & 2, and 2 for 1 & 2
    #this risk is deemed to be very low, but we could adapt this function to make a '3' category in case of tie
    return int(bldgnum)

In [32]:
#test
bldgguess(cleandata2.iloc[0,474:477])
#fantabular

1

In [33]:
#apply method to get all
cleandata2.iloc[0:4,474:477].apply(bldgguess,axis=1)

0    1
1    1
2    1
3    1
dtype: int64

In [34]:
#so, let's see if we can do this on the whole enchilada

#run this first if rerunning
#cleandata3=cleandata2.iloc[:,0:477].copy()

#modified below cleandata3->cleandata2 to bypass debug code above
#was helpful in testing, when rerunning, but now that there is working code not needed
cleandata4=pd.concat((cleandata2, cleandata2.iloc[:,474:477].apply(bldgguess,axis=1).rename('bldgguess')), axis=1).copy()
print(cleandata4.head(6))

#clear variable
cleandata2 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012    ...      BUILDINGID  SPACEID  RELATIVEPOSITION  USERID  PHONEID  \
0     100    ...               1      106                 2       2       23   
1     100    ...               1      106                 2       2       23   
2     100    ...               1      103                 2       2       23   
3     100    ...               1      102                 2       2       23   
5     100    ...               1      105                 2 

In [35]:
#now, some % of total variables
#to avoid the hassle of tuple unpacking contortions nonsense, will call it 3 times
#and python will just have to suffer through doing the sum 3 times

#start with a function
def bldgratios(observation, bldg):
    var = {0:'bldg0sum', 1:'bldg1sum', 2:'bldg2sum'}
    return (observation[var[bldg]]/(observation[var[0]]+observation[var[1]]+observation[var[2]]))

In [36]:
#make enchiladas
#make them spicy/risky by trying apply with extra arg
cleandata5=pd.concat((cleandata4, cleandata4.iloc[:,474:477].apply(bldgratios,axis=1, bldg=0).rename('bldg0ratio')), axis=1).copy()
cleandata6=pd.concat((cleandata5, cleandata5.iloc[:,474:477].apply(bldgratios,axis=1, bldg=1).rename('bldg1ratio')), axis=1).copy()
cleandata7=pd.concat((cleandata6, cleandata6.iloc[:,474:477].apply(bldgratios,axis=1, bldg=2).rename('bldg2ratio')), axis=1).copy()
print(cleandata7.head(6))
#mondoriffic, first try!
#I mean, yeah, all the ones above were first try too.  Brilliant

#clear variables
cleandata4 = None
cleandata5 = None
cleandata6 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012     ...      USERID  PHONEID   TIMESTAMP  bldg0sum  bldg1sum  \
0     100     ...           2       23  1371713733       469     15041   
1     100     ...           2       23  1371713691       367     13582   
2     100     ...           2       23  1371714095      1100     13400   
3     100     ...           2       23  1371713807      2040     13486   
5     100     ...           2       23  1371713841      1623     11490   
6     100     ..

In [37]:
#one more ratio type
#I can't effectively explain why I want to add this one, but the same reason having a normalized ratio is important
#over just the sums, we need something similar for the ratios themselves to give force to the degree the winner exceeds the other
#I expect that in the learning model will interact with bldgguess
#that is, if bldgguess doesn't win outright anyway
#what it is is the degree to which it exceeds the next one
def bldgsuperratio(observation):
    var = {0:'bldg0sum', 1:'bldg1sum', 2:'bldg2sum'}  #as always, copy what you can to make life easier
    maxone = max(observation[var[0]], observation[var[1]],observation[var[2]])
    medone = max(min(observation[var[0]], observation[var[1]]),min(observation[var[1]],observation[var[2]]), min(observation[var[0]],observation[var[2]]))
    #above line shows how cleverness, even to make a succinct 1 line, often sacrifices readibility
    #three pairs of mins will always not have the max, but the min(max, second max) will return the second max, ie median
    #which is what we want.  max(min, min, median) will get us the median
    #we could have also done sum(all 3) minus max() minus min() but I think that would have been even longer
    
    #now, we need to work around if there is no runner up
    #lets do arbitrarily high 999
    if medone == 0:
        sratio = 999
    else:
        sratio = maxone/medone
    return sratio #returns how much the max exceeds the runner up, as a percentage, or 999 if no runner up

In [38]:
#do it

#run this first if rerunning
#cleandata8=cleandata7.iloc[:,0:481].copy()

#cleandata8->cleandata7 below
cleandata9=pd.concat((cleandata7, cleandata7.iloc[:,474:477].apply(bldgsuperratio,axis=1).rename('bldgsuperratio')), axis=1).copy()
print(cleandata9.head(6))

#clear variables
cleandata7 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012       ...        PHONEID   TIMESTAMP  bldg0sum  bldg1sum  bldg2sum  \
0     100       ...             23  1371713733       469     15041         0   
1     100       ...             23  1371713691       367     13582         0   
2     100       ...             23  1371714095      1100     13400      2599   
3     100       ...             23  1371713807      2040     13486         0   
5     100       ...             23  1371713841      1623    

In [39]:
#and hopefully that will give us enough for the learning models, and maybe then some, to predict building
#which will go super fast with our 8 variables

#imports, copied over from last classification project
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [40]:
#dependent variable
depVar = cleandata9.loc[:,'BUILDINGID'].copy()
print(depVar.head(5))

#features
features = cleandata9.iloc[:,474:482].copy()
print(features.head(5))

#make train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(features, depVar, random_state = 123) #added random state so reproduceable
print(xtrain.shape)
print(xtest.shape)

0    1
1    1
2    1
3    1
5    1
Name: BUILDINGID, dtype: int64
   bldg0sum  bldg1sum  bldg2sum  bldgguess  bldg0ratio  bldg1ratio  \
0       469     15041         0          1    0.030239    0.969761   
1       367     13582         0          1    0.026310    0.973690   
2      1100     13400      2599          1    0.064331    0.783672   
3      2040     13486         0          1    0.131393    0.868607   
5      1623     11490         0          1    0.123770    0.876230   

   bldg2ratio  bldgsuperratio  
0    0.000000       32.070362  
1    0.000000       37.008174  
2    0.151997        5.155829  
3    0.000000        6.610784  
5    0.000000        7.079482  
(14895, 8)
(4966, 8)


In [41]:
#models below
#results commented, cells removed from active processing for future reruns

#random forest
rf_clf = make_pipeline(RandomForestClassifier(random_state = 123))
param_grid = {'randomforestclassifier__n_estimators': np.arange(1, 26, 5),
             'randomforestclassifier__criterion': ["gini", "entropy"]}
rf_cv = GridSearchCV(rf_clf, param_grid, cv = 10)
%timeit rf_cv.fit(xtrain, ytrain)
print("Tuned rf parameters: {}".format(rf_cv.best_params_))
print("Best score is {}".format(rf_cv.best_score_))
#print(sorted(rf_clf.get_params().keys()))

#results
#8.78 s ± 73.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned rf parameters: {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__n_estimators': 16}
#Best score is 0.9991943605236656

#given the high accuracy on RF for building, probably don't need any more models, but want to set them up anyway
#for use at the other 3 levels of prediction
#gradient boosting
gb_clf = make_pipeline(GradientBoostingClassifier(random_state = 123))
param_grid_gb = { 
        'gradientboostingclassifier__n_estimators': [5, 20, 100], 
        'gradientboostingclassifier__learning_rate': [.05, .1, .5, 1.0],
        'gradientboostingclassifier__max_depth': [3, 5, 7]}
gb_cv = GridSearchCV(gb_clf, param_grid_gb, cv = 10)
%timeit gb_cv.fit(xtrain, ytrain)
print("Tuned gb parameters: {}".format(gb_cv.best_params_))
print("Best score is {}".format(gb_cv.best_score_))
#print(sorted(gb_clf.get_params().keys()))

#results
#3min 54s ± 333 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned gb parameters: {'gradientboostingclassifier__learning_rate': 0.1, 'gradientboostingclassifier__max_depth': 3, 'gradientboostingclassifier__n_estimators': 100}
#Best score is 0.9992614971466935

#knn
knn_clf = make_pipeline(KNeighborsClassifier())
param_grid_knn = { 
        'kneighborsclassifier__n_neighbors': [3, 5, 7, 9, 15, 25] 
                }
knn_cv = GridSearchCV(knn_clf, param_grid_knn, cv = 10)
%timeit knn_cv.fit(xtrain, ytrain)
print("Tuned knn parameters: {}".format(knn_cv.best_params_))
print("Best score is {}".format(knn_cv.best_score_))
#print(sorted(knn_clf.get_params().keys()))

#results
#7.08 s ± 28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned knn parameters: {'kneighborsclassifier__n_neighbors': 3}
#Best score is 0.99906008727761

#decision tree, allows for visualization if can figure out graphviz
dt_clf = make_pipeline(DecisionTreeClassifier(random_state = 123))
param_grid = {'decisiontreeclassifier__criterion': ["gini", "entropy"]}
dt_cv = GridSearchCV(dt_clf, param_grid, cv = 10)
%timeit dt_cv.fit(xtrain, ytrain)
print("Tuned dt parameters: {}".format(dt_cv.best_params_))
print("Best score is {}".format(dt_cv.best_score_))
#print(sorted(dt_clf.get_params().keys()))

#results
#543 ms ± 2.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned dt parameters: {'decisiontreeclassifier__criterion': 'entropy'}
#Best score is 0.998992950654582

#svc
svc_clf = make_pipeline(SVC(random_state = 123))
param_grid_svc = { 
        'svc__kernel': ['linear','rbf'],
        'svc__C': [1, 10],
                    }
svc_cv = GridSearchCV(svc_clf, param_grid_svc, cv = 10)
%timeit svc_cv.fit(xtrain, ytrain)
print("Tuned svc parameters: {}".format(svc_cv.best_params_))
print("Best score is {}".format(svc_cv.best_score_))
#print(sorted(svc_clf.get_params().keys()))

#results
#22min 8s ± 4.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned svc parameters: {'svc__C': 1, 'svc__kernel': 'linear'}
#Best score is 0.9989258140315542

In [42]:
#now models using optimal settings
modelRF = RandomForestClassifier(n_estimators=16, criterion = 'gini')
modelGB = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=100)
modelKNN = KNeighborsClassifier(n_neighbors=3)
modelDT = DecisionTreeClassifier(criterion = 'entropy')
modelSVC = SVC(C=1, kernel='linear')

In [43]:
#the fit
modelRF.fit(xtrain, ytrain)
modelGB.fit(xtrain, ytrain)
modelKNN.fit(xtrain, ytrain)
modelDT.fit(xtrain, ytrain)
modelSVC.fit(xtrain, ytrain)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
#now predict
predictionsRF = modelRF.predict(xtest)
predictionsGB = modelGB.predict(xtest)
predictionsKNN = modelKNN.predict(xtest)
predictionsDT = modelDT.predict(xtest)
predictionsSVC = modelSVC.predict(xtest)

In [45]:
#score
print(accuracy_score(ytest, predictionsRF))
print(accuracy_score(ytest, predictionsGB))
print(accuracy_score(ytest, predictionsKNN))
print(accuracy_score(ytest, predictionsDT))
print(accuracy_score(ytest, predictionsSVC))

#results
#RF 0.9991945227547322
#GB 0.9991945227547322
#KN 0.9985904148207813
#DT 0.9993958920660492
#SV 0.9987917841320982

#compare to train results
#RF 0.9991943605236656
#GB 0.9992614971466935
#KN 0.99906008727761
#DT 0.998992950654582
#SV 0.9989258140315542

0.9989931534434152
0.9991945227547322
0.9985904148207813
0.9993958920660492
0.9987917841320982


In [46]:
predictionsRF

array([2, 2, 0, ..., 2, 1, 1], dtype=int64)

In [47]:
BldgPred = modelRF.predict(features)

In [48]:
BldgPred.shape

(19861,)

In [49]:
#panda dataframe needed for ease later
BldgPredpd = pd.DataFrame({'BldgPred':BldgPred}, index = cleandata9.index)
BldgPredpd.info

<bound method DataFrame.info of        BldgPred
0             1
1             1
2             1
3             1
5             1
6             1
7             1
8             1
9             1
10            1
11            1
12            1
13            1
14            1
15            1
16            1
17            1
18            1
19            1
20            1
21            1
22            1
23            1
24            1
25            1
26            1
27            1
28            1
29            1
30            1
...         ...
19907         1
19908         1
19909         1
19910         2
19911         2
19912         2
19913         2
19914         2
19915         2
19916         2
19917         2
19918         2
19919         2
19920         2
19921         2
19922         2
19923         2
19924         2
19925         2
19926         2
19927         2
19928         2
19929         1
19930         1
19931         2
19932         1
19933         2
19934         1
19935   

In [50]:
BldgPredpd.shape

(19861, 1)

In [51]:
cleandata9.shape

(19861, 482)

In [52]:
#no huge deviations.  DT actually went up, RF stayed the same, rest dropped a little bit
#going with RF for its high accuracy, consistency between train and test, and shorter run time than GB
#then assign to prediction variable to be used for next phase, FLOOR

cleandata10=pd.concat((cleandata9, BldgPredpd.loc[:,'BldgPred']),axis=1).copy()
print(cleandata10.head(6))
print(cleandata10.shape)

#clear variable
cleandata9 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012    ...      TIMESTAMP  bldg0sum  bldg1sum  bldg2sum  bldgguess  \
0     100    ...     1371713733       469     15041         0          1   
1     100    ...     1371713691       367     13582         0          1   
2     100    ...     1371714095      1100     13400      2599          1   
3     100    ...     1371713807      2040     13486         0          1   
5     100    ...     1371713841      1623     11490         0          1   
6   

In [53]:
#out of curiosity, let's see how many predictions actually differ from the munged guess variable
guessmatch = pd.DataFrame({'Match': cleandata10.loc[:,'bldgguess'] == cleandata10.loc[:,'BldgPred']})
print(guessmatch.shape)
guessmatch.groupby('Match').size()

#results
#Match
#False      288     ~ 1.45%
#True     19573
#ie, model only changed 1.45% of values from guess
#!why does this number change slightly on reruns?

(19861, 1)


Match
False      289
True     19572
dtype: int64

In [54]:
#and how many predictions actually differ from the correct
predictmatch = pd.DataFrame({'Match': cleandata10.loc[:,'BUILDINGID'] == cleandata10.loc[:,'BldgPred']})
print(predictmatch.shape)
predictmatch.groupby('Match').size()

#results
#Match
#False        4     ~ 0%
#True     19857
#not too shabby
#!why does this number change slightly on reruns?
#oh, fixed random_state on floor, but probably didn't come back to fix it here

(19861, 1)


Match
False        5
True     19856
dtype: int64

In [55]:
#now for the next level, FLOOR
floordata.groupby('FLOOR').agg('count')
#there are up to 4 floors for each building

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519
FLOOR,,,,,,,,,,,,,,,,,,,,,
0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [56]:
#this one I am going to process just like the building one
#the difference is that a particular floor will be different depending on building
#however, given that we have already determined building, there will only be unique floors
#also, our methodology of simple counting, given a building has been determined, means that the modeling
#will be blind to specific building/floor, and will apply/work equally
#I will make variables/functions like we did for building
#only differences will be a) 4 floors instead of 3 buildings, and 
#b) possible tweaks to calling the data tables because of multiple hierarchy levels

In [57]:
print(floordata)
print(floordata.shape)

                  WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  \
BUILDINGID FLOOR                                                           
0          0           0       0       0       0     246      18      25   
           1           0       0       0       0     209      46      90   
           2           5       9       0       0     119      19     340   
           3          13      10       0       0       4      38     140   
1          0           0       0       0     226       0     160       0   
           1           0       0       0      26       0     336       0   
           2           0       0       0      28       0      60       0   
           3           0       0       0       0       0       0       0   
2          0           0       0       0      22       0       0       0   
           1           0       0       0       4       0       0       0   
           2           0       0      17       0       0       0       0   
           3

In [58]:
floordata[applWAPs(cleandata.iloc[0,0:465])]

WAP090  WAP091  WAP103  WAP104  WAP105  WAP106  WAP125  \
BUILDINGID FLOOR                                                           
0          0           0       0       6       5       0       0       0   
           1           0       0       0       0       0       0       0   
           2           0       0       0       0       0       0       0   
           3           0       0       0       0       0       0       0   
1          0          48      65     163     158     140     137     206   
           1         416     390     504     591     553     547     499   
           2         346     349     230     232     207     204      81   
           3         196     208      38      41      26      51       3   
2          0           0       0       0       0       0       0       0   
           1           0       0       0       0       0       0       0   
           2           0       0       0       0       0       0       0   
           3           0       0       0       0       0       0       0   
           4           0       0       0       0       0       0       0   

                  WAP126  WAP166  WAP167  WAP168  WAP169  WAP171  WAP172  \
BUILDINGID FLOOR                                                           
0          0           0      43      38       0       0       0      26   
           1           0       5      11       0       3       0     123   
           2           0       2       3       0       3       1      14   
           3           0       5       5       3       0       0       0   
1          0         213     106     107       2       0      66      42   
           1         503     552     548      67      75     377     484   
           2          80     327     348     224     230     327     283   
           3           2     100     103     361     361     110     220   
2          0           0       0       0       0       0       0       0   
           1           0       0       0       0       0       0       0   
           2           0       0       0       0       0       0       0   
           3           0       0       0       0       0       0       0   
           4           0       0       0       0       0       0       0   

                  WAP173  WAP191  WAP192  
BUILDINGID FLOOR                          
0          0          34       0       0  
           1         116       0       2  
           2           5       9       7  
           3           0       0       0  
1          0          49       0       0  
           1         482      37      28  
           2         274     248     254  
           3         238     284     300  
2          0           0       0       0  
           1           0       0       0  
           2           0       0       0  
           3           0       0       0  
           4           0       0       0

In [59]:
#applWAPs function will work the same
#we need to make a floor sum function analog to the building sum

#first, test to see if we can call the nested hierarchy manually
#floordata[applWAPs(cleandata.iloc[0,0:465])]  returns stuff
#floordata[('BldgPred' == 0, applWAPs(cleandata10.iloc[0,0:465]))]  errors
#floordata.loc[:,('BldgPred',applWAPs(cleandata10.iloc[0,0:465]))]   errors
#floordata[['BldgPred' == 0, applWAPs(cleandata10.iloc[0,0:465])]]   errors

#ookkk, let's break it down
#applWAPs(cleandata.iloc[0,0:465])   this returns a list of 17 WAPs
#floordata[0, 'WAP090']  returns KeyError: (0, 'WAP090')
#floordata.loc['BUILDINGID', 'WAP090']   returns KeyError: 'the label [BUILDINGID] is not in the [index]'
#floordata.loc[floordata.BUILDINGID == 0, 'WAP090']   returns AttributeError: 'DataFrame' object has no attribute 'BUILDINGID'
#floordata[0,0]   returns KeyError: (0, 0)
#floordata[0][0]   returns KeyError: 0
#adding floordata shape above

#next three show how internet searching is largely useless
#floordata.loc[floordata.groups['BUILDINGID' == 0],('WAP090','WAP091')]   returns AttributeError: 'DataFrame' object has no attribute 'groups'
#floordata[["WAP090", "WAP091"]].get_group("BUILDINGID")
#floordata.get_group('BUILDINGID')['WAP090']


#floordata[('BldgPred',applWAPs(cleandata.iloc[0,0:465]))].sum(axis=1)

#function to return the floordata[applWAPs(cleandata.iloc[0,0:465])].sum(axis=1) type data

#def floorsums (observation):
#    fsums = floordata[(applWAPs(observation)].sum(axis=1)
#    return bsums[0], bsums[1], bsums[2]

In [60]:
#continued testing
floordata['WAP090']

BUILDINGID  FLOOR
0           0          0
            1          0
            2          0
            3          0
1           0         48
            1        416
            2        346
            3        196
2           0          0
            1          0
            2          0
            3          0
            4          0
Name: WAP090, dtype: int32

In [61]:
floordata['WAP090'][0]

FLOOR
0    0
1    0
2    0
3    0
Name: WAP090, dtype: int32

In [62]:
floordata['WAP090'][0,0]

0

In [63]:
#one that might mean something
floordata['WAP090'][1,0]

48

In [64]:
floordata.loc[:,'WAP090'][1,0]

48

In [65]:
floordata.loc[:,'WAP090']

BUILDINGID  FLOOR
0           0          0
            1          0
            2          0
            3          0
1           0         48
            1        416
            2        346
            3        196
2           0          0
            1          0
            2          0
            3          0
            4          0
Name: WAP090, dtype: int32

In [66]:
floordata.loc[:,'WAP090'].iloc[5]

416

In [67]:
floordata['WAP090'][1]

FLOOR
0     48
1    416
2    346
3    196
Name: WAP090, dtype: int32

In [68]:
floordata['WAP090']['BUILDINGID' == 1]
#I think I may need to remake these tables, so building ID repeats

0

In [69]:
floordatatemp = WAPdata.iloc[:,0:470].drop(['LONGITUDE','LATITUDE','SPACEID'], axis=1).groupby(['BUILDINGID','FLOOR'], as_index = False).sum(axis=0).copy()
print(floordatatemp)

    BUILDINGID  FLOOR  WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  \
0            0      0       0       0       0       0     246      18      25   
1            0      1       0       0       0       0     209      46      90   
2            0      2       5       9       0       0     119      19     340   
3            0      3      13      10       0       0       4      38     140   
4            1      0       0       0       0     226       0     160       0   
5            1      1       0       0       0      26       0     336       0   
6            1      2       0       0       0      28       0      60       0   
7            1      3       0       0       0       0       0       0       0   
8            2      0       0       0       0      22       0       0       0   
9            2      1       0       0       0       4       0       0       0   
10           2      2       0       0      17       0       0       0       0   
11           2      3       

In [70]:
floordatatemp.loc[['BUILDINGID'==1,'FLOOR'==0]]

,BUILDINGID,FLOOR,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519


In [71]:
floordatatemp.loc[['BUILDINGID'==1]]

,BUILDINGID,FLOOR,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519


In [72]:
#floordatatemp.loc['BUILDINGID'==1]  #fails, key error

In [73]:
floordatatemp.loc[floordatatemp['BUILDINGID']==1]

,BUILDINGID,FLOOR,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519
4,1,0,0,0,0,226,0,160,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,26,0,336,0,40,...,0,0,0,0,0,0,0,0,0,0
6,1,2,0,0,0,28,0,60,0,30,...,0,0,0,0,0,0,0,0,0,0
7,1,3,0,0,0,0,0,0,0,17,...,0,0,0,0,0,0,0,0,0,0


In [74]:
#floordatatemp.loc[(floordatatemp['BUILDINGID']==1, floordatatemp['FLOOR']==0)]  #fails

In [75]:
floordatatemp.loc[(floordatatemp['BUILDINGID']==1) & (floordatatemp['FLOOR']==0)]

,BUILDINGID,FLOOR,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519
4,1,0,0,0,0,226,0,160,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
#so, we can call the modified one as a normal dataframe
#still, our building one is set up as the groupby one, so it would be nice to not have to redo it
#attempted some more tests on floordata that were unsuccessful, will make floordata like floordatatemp with no index

In [77]:
fltemp2 = floordatatemp.loc[floordatatemp['BUILDINGID']==1]  #have to filter to building first, couldn't figure out how to combine with vars
tempvars = applWAPs(cleandata.iloc[0,0:465])
fltemp2[tempvars]
#can't figure out how to add 'FLOOR'
#We'll do it dirty; since we know the rows are in ascending order we'll just assign them directly

,WAP090,WAP091,WAP103,WAP104,WAP105,WAP106,WAP125,WAP126,WAP166,WAP167,WAP168,WAP169,WAP171,WAP172,WAP173,WAP191,WAP192
4,48,65,163,158,140,137,206,213,106,107,2,0,66,42,49,0,0
5,416,390,504,591,553,547,499,503,552,548,67,75,377,484,482,37,28
6,346,349,230,232,207,204,81,80,327,348,224,230,327,283,274,248,254
7,196,208,38,41,26,51,3,2,100,103,361,361,110,220,238,284,300


In [78]:
fltemp2[tempvars].sum(axis=1).reset_index(drop=True)

0    1502
1    6653
2    4244
3    2642
dtype: int64

In [79]:
#function to return the floordata[applWAPs(cleandata.iloc[0,0:465])].sum(axis=1) type data
#modified, observation to include all variables, so that BldgPred is included, so apply can be used
#in R you could extend over multiple variables with apply, but I don't know how to do it in Python
#I think Python may be too finnicky for that to work

def floorsums (observation):
    bldg = observation['BldgPred']
    fltemp = floordatatemp.loc[floordatatemp['BUILDINGID']==bldg]
    fsums = fltemp[applWAPs(observation.iloc[0:465])].sum(axis=1).reset_index(drop=True)
    fsums2 = pd.DataFrame(data = fsums,index = np.arange(5))
    #if to account for buildings with 4 or 5 floors
    if bldg == 2:
        return fsums[0], fsums[1], fsums[2], fsums[3], fsums[4]
    else:
        return fsums[0], fsums[1], fsums[2], fsums[3], 0

In [80]:
#test
floorsums(cleandata10.iloc[0,0:483])
#fantabular

(1502, 6653, 4244, 2642, 0)

In [81]:
#apply method to get all
cleandata10.iloc[0:4,0:483].apply(floorsums,axis=1)

0    (1502, 6653, 4244, 2642, 0)
1    (1259, 6100, 3697, 2526, 0)
2    (1403, 5908, 3745, 2344, 0)
3    (1367, 5631, 3828, 2660, 0)
dtype: object

In [82]:
#whole enchilada
cleandata11 = pd.concat((cleandata10, cleandata10.iloc[:,0:483].apply(floorsums,axis=1).rename('temp')),axis=1).copy()
cleandata11.head(6)

#clear variable
cleandata10 = None

In [83]:
cleandata12=pd.concat((cleandata11, cleandata11.loc[:,'temp'].apply(pd.Series,index=["flr0sum","flr1sum", "flr2sum", "flr3sum", "flr4sum"])), axis=1).copy()
cleandata12.head(6)

#clear variable
cleandata11 = None

In [84]:
cleandata12.drop('temp', axis=1, inplace=True)
cleandata12.head(6)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,bldg0ratio,bldg1ratio,bldg2ratio,bldgsuperratio,BldgPred,flr0sum,flr1sum,flr2sum,flr3sum,flr4sum
0,100,100,100,100,100,100,100,100,100,100,...,0.030239,0.969761,0.000000,32.070362,1,1502,6653,4244,2642,0
1,100,100,100,100,100,100,100,100,100,100,...,0.026310,0.973690,0.000000,37.008174,1,1259,6100,3697,2526,0
2,100,100,100,100,100,-97,100,100,100,100,...,0.064331,0.783672,0.151997,5.155829,1,1403,5908,3745,2344,0
3,100,100,100,100,100,100,100,100,100,100,...,0.131393,0.868607,0.000000,6.610784,1,1367,5631,3828,2660,0
5,100,100,100,100,100,100,100,100,100,100,...,0.123770,0.876230,0.000000,7.079482,1,1017,4777,3342,2354,0
6,100,100,100,100,100,100,100,100,100,100,...,0.056113,0.805914,0.137973,5.841093,1,1349,6348,4483,3001,0


In [85]:
cleandata12.iloc[0,483:488]

flr0sum    1502.0
flr1sum    6653.0
flr2sum    4244.0
flr3sum    2642.0
flr4sum       0.0
Name: 0, dtype: float64

In [86]:
cleandata12.iloc[0,483:488].reset_index(drop = True)

0    1502.0
1    6653.0
2    4244.0
3    2642.0
4       0.0
Name: 0, dtype: float64

In [87]:
cleandata12.iloc[0,483:488].reset_index(drop = True).max()

6653.0

In [88]:
cleandata12.iloc[0,483:488].reset_index(drop = True).idxmax()

1

In [89]:
#now, to make the other munged variables for the floor learning models to use
#let's start with a function to 'guess' which floor is the winner - the max
#since we'll have 5 floors (0 through 4) our task becomes more complex than building
#lets see if we can come up with a better way

def flrguess(observation):
    return int(observation.reset_index(drop = True).idxmax())

#that is much slicker and more elegant than the building one

In [90]:
#test
flrguess(cleandata12.iloc[0,483:488])
#fantabular

1

In [91]:
#apply method to get all
cleandata12.iloc[0:4,483:488].apply(flrguess,axis=1)

0    1
1    1
2    1
3    1
dtype: int64

In [92]:
#so, let's see if we can do this on the whole enchilada

#run this first if rerunning
#cleandata13=cleandata12.iloc[:,0:488].copy()

#below 13->12
cleandata14=pd.concat((cleandata12, cleandata12.iloc[:,483:488].apply(flrguess,axis=1).rename('flrguess')), axis=1).copy()
print(cleandata14.head(6))

#clear variable
cleandata12 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012    ...     bldg1ratio  bldg2ratio  bldgsuperratio  BldgPred  \
0     100    ...       0.969761    0.000000       32.070362         1   
1     100    ...       0.973690    0.000000       37.008174         1   
2     100    ...       0.783672    0.151997        5.155829         1   
3     100    ...       0.868607    0.000000        6.610784         1   
5     100    ...       0.876230    0.000000        7.079482         1   
6     100    ...      

In [93]:
#now, some % of total variables
#to avoid the hassle of tuple unpacking contortions nonsense, will call it 5 times
#and python will just have to suffer through doing the sum 5 times

#start with a function
def flrratios(observation, flr):
    var = {0:'flr0sum', 1:'flr1sum', 2:'flr2sum', 3:'flr3sum', 4:'flr4sum'}
    return (observation[var[flr]]/(observation[var[0]]+observation[var[1]]+observation[var[2]]+observation[var[3]]+observation[var[4]]))

In [94]:
flrratios(cleandata14.iloc[0,483:488],0)

0.09986038162356227

In [95]:
#make enchiladas
cleandata15=pd.concat((cleandata14, cleandata14.iloc[:,483:488].apply(flrratios,axis=1, flr=0).rename('flr0ratio')), axis=1).copy()
cleandata16=pd.concat((cleandata15, cleandata15.iloc[:,483:488].apply(flrratios,axis=1, flr=1).rename('flr1ratio')), axis=1).copy()
cleandata17=pd.concat((cleandata16, cleandata16.iloc[:,483:488].apply(flrratios,axis=1, flr=2).rename('flr2ratio')), axis=1).copy()
cleandata18=pd.concat((cleandata17, cleandata17.iloc[:,483:488].apply(flrratios,axis=1, flr=3).rename('flr3ratio')), axis=1).copy()
cleandata19=pd.concat((cleandata18, cleandata18.iloc[:,483:488].apply(flrratios,axis=1, flr=4).rename('flr4ratio')), axis=1).copy()
print(cleandata19.head(6))

#clear variables
cleandata14 = None
cleandata15 = None
cleandata16 = None
cleandata17 = None
cleandata18 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012    ...      flr1sum  flr2sum  flr3sum  flr4sum  flrguess  flr0ratio  \
0     100    ...         6653     4244     2642        0         1   0.099860   
1     100    ...         6100     3697     2526        0         1   0.092696   
2     100    ...         5908     3745     2344        0         1   0.104701   
3     100    ...         5631     3828     2660        0         1   0.101364   
5     100    ...         4777     3342     2354        

In [96]:
cleandata19.iloc[0,483:488].reset_index(drop = True)

0    1502.0
1    6653.0
2    4244.0
3    2642.0
4       0.0
Name: 0, dtype: float64

In [97]:
cleandata19.iloc[0,483:488].reset_index(drop = True).nlargest(2)

1    6653.0
2    4244.0
Name: 0, dtype: float64

In [98]:
cleandata19.iloc[0,483:488].reset_index(drop = True).nlargest(2)[2]

4244.0

In [99]:
cleandata19.iloc[0,483:488].reset_index(drop = True).nlargest(2).min()

4244.0

In [100]:
cleandata19.iloc[0,483:488].reset_index(drop = True).nlargest(2).index[1]

2

In [101]:
#floor's superratio
#like floor's guess, let's try to do this one better than building's

def flrsuperratio(observation):
    maxone = observation.reset_index(drop = True).nlargest(2).max()
    medone = observation.reset_index(drop = True).nlargest(2).min()
    #much better than building one, don't even need vars  

    #now, we need to work around if there is no runner up
    #lets do arbitrarily high 999
    if medone == 0:
        sratio = 999
    else:
        sratio = maxone/medone
    return sratio #returns how much the max exceeds the runner up, as a percentage, or 999 if no runner up    

In [102]:
#do it

#run this first if rerunning
#cleandata20=cleandata19.iloc[:,0:494].copy()

#20->19
cleandata21=pd.concat((cleandata19, cleandata19.iloc[:,483:488].apply(flrsuperratio,axis=1).rename('flrsuperratio')), axis=1).copy()
print(cleandata21.head(6))

#clear variable
cleandata19 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012      ...        flr2sum  flr3sum  flr4sum  flrguess  flr0ratio  \
0     100      ...           4244     2642        0         1   0.099860   
1     100      ...           3697     2526        0         1   0.092696   
2     100      ...           3745     2344        0         1   0.104701   
3     100      ...           3828     2660        0         1   0.101364   
5     100      ...           3342     2354        0         1   0.088512   
6   

In [103]:
#now for floor model stuff (pun intended)

#dependent variable
depVarflr = cleandata21.loc[:,'FLOOR'].copy()

#features
featuresflr = cleandata21.iloc[:,483:495].copy()
print(featuresflr.head(6))

#make train and test sets
fxtrain, fxtest, fytrain, fytest = train_test_split(featuresflr, depVarflr, random_state = 123) #added random state so reproduceable
print(fxtrain.shape)
print(fxtest.shape)

   flr0sum  flr1sum  flr2sum  flr3sum  flr4sum  flrguess  flr0ratio  \
0     1502     6653     4244     2642        0         1   0.099860   
1     1259     6100     3697     2526        0         1   0.092696   
2     1403     5908     3745     2344        0         1   0.104701   
3     1367     5631     3828     2660        0         1   0.101364   
5     1017     4777     3342     2354        0         1   0.088512   
6     1349     6348     4483     3001        0         1   0.088861   

   flr1ratio  flr2ratio  flr3ratio  flr4ratio  flrsuperratio  
0   0.442324   0.282162   0.175653        0.0       1.567625  
1   0.449124   0.272198   0.185981        0.0       1.649986  
2   0.440896   0.279478   0.174925        0.0       1.577570  
3   0.417544   0.283850   0.197242        0.0       1.471003  
5   0.415753   0.290862   0.204874        0.0       1.429384  
6   0.418154   0.295303   0.197681        0.0       1.416016  
(14895, 12)
(4966, 12)


In [104]:
#models below
#results will be commented, cells removed from active processing for future reruns

#random forest
rf_clf = make_pipeline(RandomForestClassifier(random_state = 123))
param_grid = {'randomforestclassifier__n_estimators': np.arange(1, 26, 5),
             'randomforestclassifier__criterion': ["gini", "entropy"]}
flrrf_cv = GridSearchCV(rf_clf, param_grid, cv = 10)
%timeit flrrf_cv.fit(fxtrain, fytrain)
print("Tuned rf parameters: {}".format(flrrf_cv.best_params_))
print("Best score is {}".format(flrrf_cv.best_score_))
#print(sorted(flrrf_clf.get_params().keys()))

#results
#42.5 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned rf parameters: {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__n_estimators': 21}
#Best score is 0.9435381000335683

#gradient boosting
gb_clf = make_pipeline(GradientBoostingClassifier(random_state = 123))
param_grid_gb = { 
        'gradientboostingclassifier__n_estimators': [5, 20, 100], 
        'gradientboostingclassifier__learning_rate': [.05, .1, .5, 1.0],
        'gradientboostingclassifier__max_depth': [3, 5, 7]}
flrgb_cv = GridSearchCV(gb_clf, param_grid_gb, cv = 10)
%timeit flrgb_cv.fit(fxtrain, fytrain)
print("Tuned gb parameters: {}".format(flrgb_cv.best_params_))
print("Best score is {}".format(flrgb_cv.best_score_))
#print(sorted(flrgb_clf.get_params().keys()))

#results
#29min 30s ± 1.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned gb parameters: {'gradientboostingclassifier__learning_rate': 0.1, 'gradientboostingclassifier__max_depth': 7, 'gradientboostingclassifier__n_estimators': 100}
#Best score is 0.9430010070493454

#knn
knn_clf = make_pipeline(KNeighborsClassifier())
param_grid_knn = { 
        'kneighborsclassifier__n_neighbors': [3, 5, 7, 9, 15, 25] 
                }
flrknn_cv = GridSearchCV(knn_clf, param_grid_knn, cv = 10)
%timeit flrknn_cv.fit(fxtrain, fytrain)
print("Tuned knn parameters: {}".format(flrknn_cv.best_params_))
print("Best score is {}".format(flrknn_cv.best_score_))
#print(sorted(flrknn_clf.get_params().keys()))

#results
#12.6 s ± 41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned knn parameters: {'kneighborsclassifier__n_neighbors': 3}
#Best score is 0.9201074185968445

#decision tree, allows for visualization if can figure out graphviz
dt_clf = make_pipeline(DecisionTreeClassifier(random_state = 123))
param_grid = {'decisiontreeclassifier__criterion': ["gini", "entropy"]}
flrdt_cv = GridSearchCV(dt_clf, param_grid, cv = 10)
%timeit flrdt_cv.fit(fxtrain, fytrain)
print("Tuned dt parameters: {}".format(flrdt_cv.best_params_))
print("Best score is {}".format(flrdt_cv.best_score_))
#print(sorted(flrdt_clf.get_params().keys()))

#results
#4.34 s ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned dt parameters: {'decisiontreeclassifier__criterion': 'entropy'}
#Best score is 0.920375965088956

#svc
svc_clf = make_pipeline(SVC(random_state = 123))
param_grid_svc = { 
        'svc__kernel': ['linear','rbf'],
        'svc__C': [1, 10],
                    }
flrsvc_cv = GridSearchCV(svc_clf, param_grid_svc, cv = 10)
%timeit flrsvc_cv.fit(fxtrain, fytrain)
print("Tuned svc parameters: {}".format(flrsvc_cv.best_params_))
print("Best score is {}".format(flrsvc_cv.best_score_))
#print(sorted(flrsvc_clf.get_params().keys()))

#results
#ran for many hours without finishing, making markdown

In [105]:
#now models using optimal settings
fmodelRF = RandomForestClassifier(n_estimators=21, criterion = 'entropy', random_state=123)
fmodelGB = GradientBoostingClassifier(learning_rate=0.1, max_depth=7, n_estimators=100, random_state=123)
fmodelKNN = KNeighborsClassifier(n_neighbors=3)
fmodelDT = DecisionTreeClassifier(criterion = 'entropy', random_state=123)
#fmodelSVC = SVC(C=1, kernel='linear', random_state=123)

In [106]:
#the fit
fmodelRF.fit(fxtrain, fytrain)
fmodelGB.fit(fxtrain, fytrain)
fmodelKNN.fit(fxtrain, fytrain)
fmodelDT.fit(fxtrain, fytrain)
#fmodelSVC.fit(fxtrain, fytrain)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [107]:
#now predict
fpredictionsRF = fmodelRF.predict(fxtest)
fpredictionsGB = fmodelGB.predict(fxtest)
fpredictionsKNN = fmodelKNN.predict(fxtest)
fpredictionsDT = fmodelDT.predict(fxtest)
#fpredictionsSVC = fmodelSVC.predict(fxtest)

In [108]:
#score
print(accuracy_score(fytest, fpredictionsRF))
print(accuracy_score(fytest, fpredictionsGB))
print(accuracy_score(fytest, fpredictionsKNN))
print(accuracy_score(fytest, fpredictionsDT))
#print(accuracy_score(fytest, fpredictionsSVC))

#results
#RF 0.9438179621425695
#GB 0.9428111155859847
#KN 0.9248892468787757
#DT 0.9283125251711639
#SV 
#!these changed slightly after rerun, why?

#compare to train results
#RF 0.9435381000335683
#GB 0.9430010070493454
#KN 0.9201074185968445
#DT 0.920375965088956
#SV 

#rf and gb are the most accurate and consistent (less chance overfit)
#like building level, will go with RF because it is vastly faster than GB
#will note here that the accuracy isn't as high as I wish it was; this is probably due to my limited number of munged variables
#will contemplate methods of adding more variables to increase accuracy, but for now will move on

0.9438179621425695
0.9428111155859847
0.9248892468787757
0.9283125251711639


In [109]:
fpredictionsRF

array([2, 0, 2, ..., 3, 2, 2], dtype=int64)

In [110]:
FlrPred = fmodelRF.predict(featuresflr)

In [111]:
FlrPred.shape

(19861,)

In [112]:
FlrPredpd = pd.DataFrame({'FlrPred':FlrPred}, index = cleandata21.index)
FlrPredpd.info

<bound method DataFrame.info of        FlrPred
0            2
1            2
2            2
3            2
5            2
6            2
7            2
8            2
9            2
10           2
11           2
12           2
13           2
14           2
15           2
16           1
17           2
18           2
19           2
20           2
21           2
22           2
23           2
24           2
25           2
26           2
27           2
28           2
29           2
30           2
...        ...
19907        3
19908        3
19909        3
19910        1
19911        1
19912        0
19913        1
19914        1
19915        1
19916        1
19917        1
19918        1
19919        1
19920        1
19921        1
19922        1
19923        1
19924        1
19925        1
19926        1
19927        1
19928        1
19929        3
19930        3
19931        1
19932        3
19933        1
19934        3
19935        3
19936        3

[19861 rows x 1 columns]>

In [113]:
FlrPredpd.shape

(19861, 1)

In [114]:
cleandata21.shape

(19861, 495)

In [115]:
#Make Floor prediction
cleandata22=pd.concat((cleandata21, FlrPredpd.loc[:,'FlrPred']),axis=1).copy()

print(cleandata21.head(6))
print(cleandata21.shape)
print(cleandata22.head(6))
print(cleandata22.shape)

#clear variable
cleandata21 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012      ...        flr2sum  flr3sum  flr4sum  flrguess  flr0ratio  \
0     100      ...           4244     2642        0         1   0.099860   
1     100      ...           3697     2526        0         1   0.092696   
2     100      ...           3745     2344        0         1   0.104701   
3     100      ...           3828     2660        0         1   0.101364   
5     100      ...           3342     2354        0         1   0.088512   
6   

In [116]:
#out of curiosity, let's see how many predictions actually differ from the munged guess variable
flrguessmatch = pd.DataFrame({'Match': cleandata22.loc[:,'flrguess'] == cleandata22.loc[:,'FlrPred']})
print(flrguessmatch.shape)
flrguessmatch.groupby('Match').size()

#results
#Match
#False     7602     ~ 38.28%
#True     12259
#ie, model changed over a third of values from guess

(19861, 1)


Match
False     7602
True     12259
dtype: int64

In [117]:
#and how many predictions actually differ from the correct
flrpredictmatch = pd.DataFrame({'Match': cleandata22.loc[:,'FLOOR'] == cleandata22.loc[:,'FlrPred']})
print(flrpredictmatch.shape)
flrpredictmatch.groupby('Match').size()

#results
#Match
#False      324     ~ 1.63%
#True     19533
#This is surprising, it is much better accuracy than showing above
#also surprising because all head values were obviously wrong
#I think something is awry
#!!!decided that the CV scores from GridSearchCV are not consistent,
#model is getting very high accuracy ~99.7% on train portion
#this indicates some overfit
#ideally, need to find a grid tune that takes in both a training set and a validation set

(19861, 1)


Match
False      324
True     19537
dtype: int64

In [118]:
#let's see
print(flrpredictmatch)

       Match
0       True
1       True
2       True
3       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15      True
16     False
17      True
18      True
19      True
20      True
21      True
22      True
23      True
24      True
25      True
26      True
27      True
28      True
29      True
30      True
...      ...
19907   True
19908   True
19909   True
19910   True
19911   True
19912  False
19913   True
19914   True
19915   True
19916   True
19917   True
19918   True
19919   True
19920   True
19921   True
19922   True
19923   True
19924   True
19925   True
19926   True
19927   True
19928   True
19929   True
19930   True
19931   True
19932   True
19933   True
19934   True
19935   True
19936   True

[19861 rows x 1 columns]


In [119]:
cleandata22[['FLOOR','FlrPred']]

,FLOOR,FlrPred
0,2,2
1,2,2
2,2,2
3,2,2
5,2,2
6,2,2
7,2,2
8,2,2
9,2,2
10,2,2


In [120]:
#ok, seems reasonable on the face of it, but -
#why is the end result so much better than the prediction scores?
#with the prediction on both train and test around 94%, you would think overall would be around 94% also
#even if that is solved, it is interesting that so many do not match the guess
#it would be nice to see what variables are actually going into the prediction
#and if the guess is that far off, I wonder if more variables should be engineered
#such as identifying WAPs that are strongly correlated with one floor, and using that as another guess
#for now, to get things to work, I will not try to create new variables
#but will try to explain why accuracy is different from expected


In [121]:
#first, let's see if we can confirm one way or the other
accuracy_score(depVarflr, FlrPred)
#result: 0.98348522229495
#matches the accuracy I calculated manually.  Why does it not match the 94%?

0.9836866220230602

In [122]:
#recap of training code
#Q: why are overall (train and test combined) results so much better than results on train and test separately?

#dependent variable
#depVarflr = cleandata21.loc[:,'FLOOR'].copy()

#features
#featuresflr = cleandata21.iloc[:,483:495].copy()

#make train and test sets
#fxtrain, fxtest, fytrain, fytest = train_test_split(featuresflr, depVarflr, random_state = 123) #added random state so reproduceable

#tune
#rf_clf = make_pipeline(RandomForestClassifier(random_state = 123))
#param_grid = {'randomforestclassifier__n_estimators': np.arange(1, 26, 5),
#             'randomforestclassifier__criterion': ["gini", "entropy"]}
#flrrf_cv = GridSearchCV(rf_clf, param_grid, cv = 10)
#%timeit flrrf_cv.fit(fxtrain, fytrain)
#print("Tuned rf parameters: {}".format(flrrf_cv.best_params_))
#print("Best score is {}".format(flrrf_cv.best_score_))
#tuning grid results:
#42.5 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned rf parameters: {'randomforestclassifiercriterion': 'entropy', 'randomforestclassifiern_estimators': 21}
#Best score is 0.9435381000335683

#now models using optimal settings
#fmodelRF = RandomForestClassifier(n_estimators=21, criterion = 'entropy', random_state=123)

#the fit
#fmodelRF.fit(fxtrain, fytrain)
#now predict
#fpredictionsRF = fmodelRF.predict(fxtest)

#print(accuracy_score(fytest, fpredictionsRF))
#results: 0.9426097462746678
#print(accuracy_score(depVarflr, FlrPred))
#results: 0.98348522229495

#from sklearn.model_selection import cross_val_score
#print(cross_val_score(fmodelRF, fxtrain, fytrain))
#results: [0.93778941 0.93836858 0.93753778]
#cross_val results similar (but not same?) to tuned grid results

#training accuracy
#fpredRFtemp = fmodelRF.predict(fxtrain)
#print(accuracy_score(fytrain, fpredRFtemp))
#results: 0.997113125209802
#so looks like the cross val score returned by GridSearchCV is inaccurate, and is hiding this overfit
#overfit makes sense, because we do not have an actual validation set, just training and test
#its possible that other parameters would have less overfit, but do not know how to do that with GridSearchCV
#if I have time, I'll try to come back and find a better tuning method, or try parameters manually (sigh)
#but alas, need to move on right now.  We'll just have to live with the overfit of 5% more accuracy on the first 75%

In [123]:
#moving on to SPACEID
#the difference in SPACEID compared to BUILDINGID, FLOOR, and RELATIVEPOSITION is that it has so many more possibilities
cleandata22.iloc[:,469].drop_duplicates().shape
#for the other 3 variables, we are making 2 sets of variables for each possibility (Building 3, Floor 5, RelPos 2)
#but 123 is too many to do that on.  While it would still be less complexity than feeding each WAP in
#it would still be too complex I think
#so for this one, we'll come up with a different set of metrics
#remember, the methodology should be 'blind' to the precursor variables, building and floor
#but I'd like to stick to a similar method, counting of possibilities
#so maybe return top 3 or 5, their % of total (or of top 3/5) and a superratio between top and runner up?

#we'll need to be careful with features if we want to include building prediction because it is noncontiguous
#with what we'll add (floor prediction is at end of floor)


(123,)

In [124]:
#at this point, memory had ballooned to about 2 and a half gigs
#!now on cleaning and rerun, memory is at about a gig

In [125]:
#first, the roomdata table, with as_index = False
roomdata = WAPdata.iloc[:,0:470].drop(['LONGITUDE','LATITUDE'], axis=1).groupby(['BUILDINGID','FLOOR','SPACEID'], as_index = False).sum(axis=0).copy()
print(roomdata)

     BUILDINGID  FLOOR  SPACEID  WAP001  WAP002  WAP005  WAP006  WAP007  \
0             0      0      102       0       0       0       0       0   
1             0      0      106       0       0       0       0       0   
2             0      0      107       0       0       0       0       0   
3             0      0      110       0       0       0       0       0   
4             0      0      111       0       0       0       0       0   
5             0      0      112       0       0       0       0       0   
6             0      0      113       0       0       0       0       0   
7             0      0      114       0       0       0       0       0   
8             0      0      115       0       0       0       0       0   
9             0      0      116       0       0       0       0       0   
10            0      0      117       0       0       0       0       0   
11            0      0      118       0       0       0       0       0   
12            0      0   

In [126]:
roomdata.loc[roomdata['BUILDINGID']==1]

,BUILDINGID,FLOOR,SPACEID,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519
256,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
257,1,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258,1,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259,1,0,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260,1,0,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261,1,0,15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
262,1,0,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263,1,0,107,0,0,0,0,0,20,0,...,0,0,0,0,0,0,0,0,0,0
264,1,0,108,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265,1,0,110,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
roomdata.loc[(roomdata['BUILDINGID']==1) & (roomdata['FLOOR']==0)]

,BUILDINGID,FLOOR,SPACEID,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519
256,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
257,1,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258,1,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259,1,0,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260,1,0,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261,1,0,15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
262,1,0,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263,1,0,107,0,0,0,0,0,20,0,...,0,0,0,0,0,0,0,0,0,0
264,1,0,108,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265,1,0,110,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
roomdata.loc[(roomdata['BUILDINGID']==1) & (roomdata['FLOOR']==0) & (roomdata['SPACEID']==107)]

,BUILDINGID,FLOOR,SPACEID,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519
263,1,0,107,0,0,0,0,0,20,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
roomtemp = roomdata.loc[(roomdata['BUILDINGID']==1) & (roomdata['FLOOR']==0)]  
tempvars2 = applWAPs(cleandata22.iloc[0,0:465])
#before we knew floors were in order, so reseting the index worked
#but now, we need SPACEID
#figured out how to add SPACEID (see below steps)
tempvars4 = tempvars2.append(pd.Index(list({'SPACEID'})))
roomtemp[tempvars4]

,WAP090,WAP091,WAP103,WAP104,WAP105,WAP106,WAP125,WAP126,WAP166,WAP167,WAP168,WAP169,WAP171,WAP172,WAP173,WAP191,WAP192,SPACEID
256,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
257,0,0,11,0,0,0,11,11,0,0,0,0,0,11,0,0,0,3
258,0,11,12,11,0,0,10,20,0,0,0,0,0,7,11,0,0,6
259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9
260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10
261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15
262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16
263,0,0,8,17,3,5,20,20,0,0,0,0,0,0,0,0,0,107
264,0,0,0,0,5,2,13,12,10,9,0,0,17,0,0,0,0,108
265,0,0,0,0,0,0,0,0,20,20,0,0,0,0,0,0,0,110


In [130]:
tempvars2

Index(['WAP090', 'WAP091', 'WAP103', 'WAP104', 'WAP105', 'WAP106', 'WAP125',
       'WAP126', 'WAP166', 'WAP167', 'WAP168', 'WAP169', 'WAP171', 'WAP172',
       'WAP173', 'WAP191', 'WAP192'],
      dtype='object')

In [131]:
list(tempvars2)

['WAP090',
 'WAP091',
 'WAP103',
 'WAP104',
 'WAP105',
 'WAP106',
 'WAP125',
 'WAP126',
 'WAP166',
 'WAP167',
 'WAP168',
 'WAP169',
 'WAP171',
 'WAP172',
 'WAP173',
 'WAP191',
 'WAP192']

In [132]:
type(tempvars2)

pandas.core.indexes.base.Index

In [133]:
tempvars3 = tempvars2.append(pd.Index(list({'SPACEID'})))
tempvars3

Index(['WAP090', 'WAP091', 'WAP103', 'WAP104', 'WAP105', 'WAP106', 'WAP125',
       'WAP126', 'WAP166', 'WAP167', 'WAP168', 'WAP169', 'WAP171', 'WAP172',
       'WAP173', 'WAP191', 'WAP192', 'SPACEID'],
      dtype='object')

In [134]:
roomtemp[tempvars4].groupby(['SPACEID']).agg('sum',axis=1).sum(axis=1)

SPACEID
1        0
3       44
6       82
9        0
10       0
15       0
16       0
107     73
108     68
110     40
111     46
112     43
113     53
114     83
115    146
116    145
117    143
118    145
119    138
121    103
122    139
202      0
203      0
213      0
216      0
217      0
218      0
219      0
220      0
222      0
224      0
225      0
226      0
227      0
228      0
229      0
230      0
231     11
235      0
237      0
238      0
239      0
243      0
244      0
245      0
246      0
247      0
248      0
249      0
dtype: int64

In [135]:
roomtemp[tempvars4].groupby(['SPACEID']).agg('sum',axis=1).sum(axis=1).nlargest(3)

SPACEID
115    146
116    145
118    145
dtype: int64

In [136]:
roomtemp[tempvars4].groupby(['SPACEID']).agg('sum',axis=1).sum(axis=1).sum()

1502

In [137]:
#function to return the roomdata
#this one is different, as it will return the top 3 rooms, and their percentage of total
#these 3 will be our 'algorithms', then we will calculate which 'algorithm' is correct
#and the models will predict algorithm
#then based on that, we'll pull the appropriate room

def roomsums (observation):
    bldg = observation['BldgPred']
    flr = observation['FlrPred']
    roomtemp2 = roomdata[(roomdata['BUILDINGID']==bldg) & (roomdata['FLOOR']==flr)]
    tempvars5 = applWAPs(observation.iloc[0:465])
    tempvars6 = tempvars5.append(pd.Index(list({'SPACEID'})))
    rsums = roomtemp2[tempvars6].groupby(['SPACEID']).agg('sum',axis=1).sum(axis=1).nlargest(3)
    rsums2 = pd.DataFrame(data = rsums)
    total = roomtemp2[tempvars6].groupby(['SPACEID']).agg('sum',axis=1).sum(axis=1).sum()
    return rsums2.iloc[[0]].index[0], rsums2.iloc[0].max(), rsums2.iloc[[1]].index[0], rsums2.iloc[1].max(), rsums2.iloc[[2]].index[0], rsums2.iloc[2].max(), total

In [138]:
cleandata22.shape

(19861, 496)

In [139]:
tempz = pd.DataFrame(roomtemp[tempvars4].groupby(['SPACEID']).agg('sum',axis=1).sum(axis=1).nlargest(3))
tempz

,0
SPACEID,
115,146
116,145
118,145


In [140]:
tempz.iloc[0].max()

146

In [141]:
tempz.iloc[0]+1

0    147
Name: 115, dtype: int64

In [142]:
tempz.iloc[[0]].index[0]

115

In [143]:
tempz.iloc[1].max()

145

In [144]:
tempz.iloc[[1]].index[0]

116

In [145]:
#test
roomsums(cleandata22.iloc[0,0:496])
#fantabular

(106, 706, 101, 687, 103, 562, 4244)

In [146]:
#apply method to get all
cleandata22.iloc[0:4,0:496].apply(roomsums,axis=1)

0    (106, 706, 101, 687, 103, 562, 4244)
1    (106, 655, 101, 592, 103, 444, 3697)
2    (106, 597, 101, 588, 103, 537, 3745)
3    (101, 584, 106, 560, 103, 509, 3828)
dtype: object

In [147]:
#whole enchilada
cleandata23 = pd.concat((cleandata22, cleandata22.iloc[:,0:496].apply(roomsums,axis=1).rename('temp')),axis=1).copy()
print(cleandata23.head(6))

#clear variable
cleandata22 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012                  ...                   flr4sum  flrguess  flr0ratio  \
0     100                  ...                         0         1   0.099860   
1     100                  ...                         0         1   0.092696   
2     100                  ...                         0         1   0.104701   
3     100                  ...                         0         1   0.101364   
5     100                  ...                         

In [148]:
cleandata24=pd.concat((cleandata23, cleandata23.loc[:,'temp'].apply(pd.Series,index=["rooma","roomasum", "roomb", "roombsum", "roomc", "roomcsum", "roomtotal"])), axis=1).copy()
print(cleandata24.head(6))

#clear variable
cleandata23 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012    ...      flrsuperratio  FlrPred  \
0     100    ...           1.567625        2   
1     100    ...           1.649986        2   
2     100    ...           1.577570        2   
3     100    ...           1.471003        2   
5     100    ...           1.429384        2   
6     100    ...           1.416016        2   

                                   temp  rooma  roomasum  roomb  roombsum  \
0  (106, 706, 101, 687, 103, 562, 4244)    106

In [149]:
cleandata24.drop('temp', axis=1, inplace=True)
cleandata24.head(6)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,flr4ratio,flrsuperratio,FlrPred,rooma,roomasum,roomb,roombsum,roomc,roomcsum,roomtotal
0,100,100,100,100,100,100,100,100,100,100,...,0.0,1.567625,2,106,706,101,687,103,562,4244
1,100,100,100,100,100,100,100,100,100,100,...,0.0,1.649986,2,106,655,101,592,103,444,3697
2,100,100,100,100,100,-97,100,100,100,100,...,0.0,1.577570,2,106,597,101,588,103,537,3745
3,100,100,100,100,100,100,100,100,100,100,...,0.0,1.471003,2,101,584,106,560,103,509,3828
5,100,100,100,100,100,100,100,100,100,100,...,0.0,1.429384,2,106,594,101,568,103,496,3342
6,100,100,100,100,100,100,100,100,100,100,...,0.0,1.416016,2,106,697,101,688,103,583,4483


In [270]:
cleandata25 = cleandata24.copy()

In [271]:
cleandata25['roomaratio']=cleandata25['roomasum']/cleandata25['roomtotal']
cleandata25['roombratio']=cleandata25['roombsum']/cleandata25['roomtotal']
cleandata25['roomcratio']=cleandata25['roomcsum']/cleandata25['roomtotal']
cleandata25.head(6)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,rooma,roomasum,roomb,roombsum,roomc,roomcsum,roomtotal,roomaratio,roombratio,roomcratio
0,100,100,100,100,100,100,100,100,100,100,...,106,706,101,687,103,562,4244,0.166352,0.161876,0.132422
1,100,100,100,100,100,100,100,100,100,100,...,106,655,101,592,103,444,3697,0.177171,0.160130,0.120097
2,100,100,100,100,100,-97,100,100,100,100,...,106,597,101,588,103,537,3745,0.159413,0.157009,0.143391
3,100,100,100,100,100,100,100,100,100,100,...,101,584,106,560,103,509,3828,0.152560,0.146290,0.132968
5,100,100,100,100,100,100,100,100,100,100,...,106,594,101,568,103,496,3342,0.177738,0.169958,0.148414
6,100,100,100,100,100,100,100,100,100,100,...,106,697,101,688,103,583,4483,0.155476,0.153469,0.130047


In [272]:
#return which of three rooms, if any, is correct

def roomwinner(observation):
    if observation['rooma']==observation['SPACEID']:
        temp = "rooma"
    elif observation['roomb']==observation['SPACEID']:
        temp = "roomb"
    elif observation['roomc']==observation['SPACEID']:
        temp = "roomc"
    else:
        temp = "none"        
    return temp

In [273]:
#test
roomwinner(cleandata25.iloc[0,466:504])
#fantabular

'rooma'

In [274]:
#apply method to get all
cleandata25.iloc[0:9,466:504].apply(roomwinner,axis=1)

0    rooma
1    rooma
2    roomc
3     none
5     none
6    roomc
7    roomb
8    roomb
9    rooma
dtype: object

In [275]:
cleandata25[['SPACEID','rooma','roomb','roomc']]

,SPACEID,rooma,roomb,roomc
0,106,106,101,103
1,106,106,101,103
2,103,106,101,103
3,102,101,106,103
5,105,106,101,103
6,103,106,101,103
7,101,106,101,103
8,112,106,112,101
9,103,103,101,106
10,104,104,103,106


In [276]:
#create winner
cleandata26=pd.concat((cleandata25, cleandata25.iloc[:,465:504].apply(roomwinner,axis=1).rename('roomwinner')), axis=1).copy()
print(cleandata26.head(6))

#clear variable
cleandata25 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012     ...      roomasum  roomb  roombsum  roomc  roomcsum  roomtotal  \
0     100     ...           706    101       687    103       562       4244   
1     100     ...           655    101       592    103       444       3697   
2     100     ...           597    101       588    103       537       3745   
3     100     ...           584    106       560    103       509       3828   
5     100     ...           594    101       568    103     

In [277]:
#previously called roomguess, but decided winner was more accurate
#commented out as will not be needed for future reruns, but nice to have the rename method here
#cleandata26.rename(columns = {'roomguess':'roomwinner'}, inplace = True)
#print(cleandata26.head(6))

In [278]:
#so the difference in this one is that we are going to guess 'roomwinner' based on existing variables
#this instead of predicting SPACEID directly
#then, after calculating a predicted room, we will make a predicted space by pulling the appropriate column's space
#I expect this one to be less accurate/have more overfit
#If time permits, more variables, and different approaches, would be nice to have
#Different approaches would make fitting work better
#It is key that each approach recommends a different SPACEID, otherwise there would be multiple winners
#and I wouldn't know how to mess with that, throws off prediction

In [279]:
#now for room model stuff

#dependent variable
depVarroom = cleandata26.loc[:,'roomwinner'].copy()

#features
featuresroom = cleandata26.iloc[:,496:506].copy()
print(featuresroom.head(6))

#make train and test sets
rmxtrain, rmxtest, rmytrain, rmytest = train_test_split(featuresroom, depVarroom, random_state = 123) #added random state so reproduceable
print(rmxtrain.shape)
print(rmxtest.shape)

   rooma  roomasum  roomb  roombsum  roomc  roomcsum  roomtotal  roomaratio  \
0    106       706    101       687    103       562       4244    0.166352   
1    106       655    101       592    103       444       3697    0.177171   
2    106       597    101       588    103       537       3745    0.159413   
3    101       584    106       560    103       509       3828    0.152560   
5    106       594    101       568    103       496       3342    0.177738   
6    106       697    101       688    103       583       4483    0.155476   

   roombratio  roomcratio  
0    0.161876    0.132422  
1    0.160130    0.120097  
2    0.157009    0.143391  
3    0.146290    0.132968  
5    0.169958    0.148414  
6    0.153469    0.130047  
(14895, 10)
(4966, 10)


In [280]:
#models below
#results will be commented, cells removed from active processing for future reruns

#random forest
rf_clf = make_pipeline(RandomForestClassifier(random_state = 123))
param_grid = {'randomforestclassifier__n_estimators': np.arange(1, 26, 5),
             'randomforestclassifier__criterion': ["gini", "entropy"]}
roomrf_cv = GridSearchCV(rf_clf, param_grid, cv = 10)
%timeit roomrf_cv.fit(rmxtrain, rmytrain)
print("Tuned rf parameters: {}".format(roomrf_cv.best_params_))
print("Best score is {}".format(roomrf_cv.best_score_))
#print(sorted(roomrf_clf.get_params().keys()))

#results
#49.3 s ± 76.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned rf parameters: {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__n_estimators': 21}
#Best score is 0.783484390735146

#gradient boosting
gb_clf = make_pipeline(GradientBoostingClassifier(random_state = 123))
param_grid_gb = { 
        'gradientboostingclassifier__n_estimators': [5, 20, 100], 
        'gradientboostingclassifier__learning_rate': [.05, .1, .5, 1.0],
        'gradientboostingclassifier__max_depth': [3, 5, 7]}
roomgb_cv = GridSearchCV(gb_clf, param_grid_gb, cv = 10)
%timeit roomgb_cv.fit(rmxtrain, rmytrain)
print("Tuned gb parameters: {}".format(roomgb_cv.best_params_))
print("Best score is {}".format(roomgb_cv.best_score_))
#print(sorted(roomgb_clf.get_params().keys()))

#results
#19min 12s ± 796 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned gb parameters: {'gradientboostingclassifier__learning_rate': 0.1, 'gradientboostingclassifier__max_depth': 7, 'gradientboostingclassifier__n_estimators': 100}
#Best score is 0.7836186639812017

#knn
knn_clf = make_pipeline(KNeighborsClassifier())
param_grid_knn = { 
        'kneighborsclassifier__n_neighbors': [3, 5, 7, 9, 15, 25] 
                }
roomknn_cv = GridSearchCV(knn_clf, param_grid_knn, cv = 10)
%timeit roomknn_cv.fit(rmxtrain, rmytrain)
print("Tuned knn parameters: {}".format(roomknn_cv.best_params_))
print("Best score is {}".format(roomknn_cv.best_score_))
#print(sorted(roomknn_clf.get_params().keys()))

#results
#9.84 s ± 16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned knn parameters: {'kneighborsclassifier__n_neighbors': 3}
#Best score is 0.6436388049681101

#decision tree, allows for visualization if can figure out graphviz
dt_clf = make_pipeline(DecisionTreeClassifier(random_state = 123))
param_grid = {'decisiontreeclassifier__criterion': ["gini", "entropy"]}
roomdt_cv = GridSearchCV(dt_clf, param_grid, cv = 10)
%timeit roomdt_cv.fit(rmxtrain, rmytrain)
print("Tuned dt parameters: {}".format(roomdt_cv.best_params_))
print("Best score is {}".format(roomdt_cv.best_score_))
#print(sorted(roomdt_clf.get_params().keys()))

#results
#4.4 s ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned dt parameters: {'decisiontreeclassifier__criterion': 'entropy'}
#Best score is 0.7319234642497482

In [281]:
#pause, to hopefully let decision tree unstar while svc is still going

#svc
svc_clf = make_pipeline(SVC(random_state = 123))
param_grid_svc = { 
        'svc__kernel': ['linear','rbf'],
        'svc__C': [1, 10],
                    }
roomsvc_cv = GridSearchCV(svc_clf, param_grid_svc, cv = 10)
%timeit roomsvc_cv.fit(rmxtrain, rmytrain)
print("Tuned svc parameters: {}".format(roomsvc_cv.best_params_))
print("Best score is {}".format(roomsvc_cv.best_score_))
#print(sorted(roomsvc_clf.get_params().keys()))

#results
#ran for many hours without finishing, making markdown

In [282]:
#now models using optimal settings
rmmodelRF = RandomForestClassifier(n_estimators=21, criterion = 'gini', random_state=123)
rmmodelGB = GradientBoostingClassifier(learning_rate=0.1, max_depth=7, n_estimators=100, random_state=123)
rmmodelKNN = KNeighborsClassifier(n_neighbors=3)
rmmodelDT = DecisionTreeClassifier(criterion = 'entropy', random_state=123)
#rmmodelSVC = SVC(C=1, kernel='linear', random_state=123)

In [283]:
#the fit
rmmodelRF.fit(rmxtrain, rmytrain)
rmmodelGB.fit(rmxtrain, rmytrain)
rmmodelKNN.fit(rmxtrain, rmytrain)
rmmodelDT.fit(rmxtrain, rmytrain)
#rmmodelSVC.fit(rmxtrain, rmytrain)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [284]:
#now predict
rmpredictionsRF = rmmodelRF.predict(rmxtest)
rmpredictionsGB = rmmodelGB.predict(rmxtest)
rmpredictionsKNN = rmmodelKNN.predict(rmxtest)
rmpredictionsDT = rmmodelDT.predict(rmxtest)
#rmpredictionsSVC = rmmodelSVC.predict(rmxtest)

In [285]:
#score
print(accuracy_score(rmytest, rmpredictionsRF))
print(accuracy_score(rmytest, rmpredictionsGB))
print(accuracy_score(rmytest, rmpredictionsKNN))
print(accuracy_score(rmytest, rmpredictionsDT))
#print(accuracy_score(rmytest, rmpredictionsSVC))

#results
#RF 0.7855416834474426
#GB 0.7877567458719291
#KN 0.657470801449859
#DT 0.7378171566653242
#SV 

#compare to train results
#RF 0.783484390735146
#GB 0.7836186639812017
#KN 0.6436388049681101
#DT 0.7319234642497482
#SV 

#rf and gb are the most accurate and consistent (less chance overfit)
#like building level, will go with RF because it is vastly faster than GB
#will note here that the accuracy isn't as high as I wish it was; this is probably due to my limited number of munged variables
#will contemplate methods of adding more variables to increase accuracy, but for now will move on

0.7855416834474426
0.7877567458719291
0.657470801449859
0.7378171566653242


In [286]:
rmpredictionsRF

array(['none', 'rooma', 'rooma', ..., 'none', 'none', 'none'],
      dtype=object)

In [287]:
RoomPred = rmmodelRF.predict(featuresroom)

In [288]:
RoomPred.shape

(19861,)

In [289]:
RoomPredpd = pd.DataFrame({'RoomPred':RoomPred}, index = cleandata26.index)
RoomPredpd.info

<bound method DataFrame.info of       RoomPred
0        rooma
1        rooma
2        roomc
3         none
5         none
6        roomc
7        roomb
8         none
9        rooma
10       rooma
11       rooma
12       rooma
13       rooma
14       roomb
15       rooma
16       roomb
17        none
18        none
19        none
20        none
21        none
22        none
23        none
24        none
25        none
26       rooma
27       rooma
28       roomb
29       rooma
30       rooma
...        ...
19907    roomb
19908    roomb
19909     none
19910     none
19911     none
19912     none
19913     none
19914     none
19915    rooma
19916     none
19917     none
19918     none
19919     none
19920     none
19921     none
19922     none
19923     none
19924    rooma
19925    roomb
19926    roomc
19927     none
19928     none
19929    roomb
19930     none
19931    roomc
19932    rooma
19933     none
19934     none
19935    roomc
19936     none

[19861 rows x 1 columns]>

In [290]:
RoomPredpd.shape

(19861, 1)

In [291]:
cleandata26.shape

(19861, 507)

In [292]:
#Make room prediction
cleandata27=pd.concat((cleandata26, RoomPredpd.loc[:,'RoomPred']),axis=1).copy()

print(cleandata26.head(6))
print(cleandata26.shape)
print(cleandata27.head(6))
print(cleandata27.shape)

#clear variable
cleandata26 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012     ...      roomasum  roomb  roombsum  roomc  roomcsum  roomtotal  \
0     100     ...           706    101       687    103       562       4244   
1     100     ...           655    101       592    103       444       3697   
2     100     ...           597    101       588    103       537       3745   
3     100     ...           584    106       560    103       509       3828   
5     100     ...           594    101       568    103     

In [293]:
#return which of three rooms, if any, is correct

def spacepred(observation):
    if observation['RoomPred'] == "rooma":
        temp = observation['rooma']
    elif observation['RoomPred'] == "roomb":
        temp = observation['roomb']
    elif observation['RoomPred'] == "roomc":
        temp = observation['roomc']
    else:
        temp = "none"        
    return temp

In [294]:
#test
spacepred(cleandata27.iloc[0,496:508])
#fantabular

106

In [295]:
#apply method to get all
cleandata27.iloc[0:9,496:508].apply(spacepred,axis=1)

0     106
1     106
2     103
3    none
5    none
6     103
7     101
8    none
9     103
dtype: object

In [296]:
#add space prediction from room prediction
cleandata28=pd.concat((cleandata27, cleandata27.iloc[:,496:508].apply(spacepred,axis=1).rename('SpacePred')), axis=1).copy()
print(cleandata28.head(6))

#clear variable
cleandata27 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012    ...      roombsum  roomc  roomcsum  roomtotal  roomaratio  \
0     100    ...           687    103       562       4244    0.166352   
1     100    ...           592    103       444       3697    0.177171   
2     100    ...           588    103       537       3745    0.159413   
3     100    ...           560    103       509       3828    0.152560   
5     100    ...           568    103       496       3342    0.177738   
6     100    ...

In [297]:
#how many space predictions actually differ from the correct

spacepredictmatch = pd.DataFrame({'Match': cleandata28.loc[:,'SPACEID'] == cleandata28.loc[:,'SpacePred']})
print(spacepredictmatch.shape)
spacepredictmatch.groupby('Match').size()

#results
#Match
#False      12322     ~ 62.04%
#True     7539
#This is quite low, and lower than expected from running the model

(19861, 1)


Match
False    12322
True      7539
dtype: int64

In [298]:
#how many room predictions actually differ from the correct

roompredictmatch = pd.DataFrame({'Match': cleandata28.loc[:,'roomwinner'] == cleandata28.loc[:,'RoomPred']})
print(roompredictmatch.shape)
roompredictmatch.groupby('Match').size()

#results
#Match
#False     1482      ~ 7.46%
#True     18379

#so something is off

(19861, 1)


Match
False     1482
True     18379
dtype: int64

In [299]:
cleandata28[['SPACEID','rooma','roomb','roomc','roomwinner','RoomPred','SpacePred']]
#ahh, nones may be the difference between the room score and the space score

,SPACEID,rooma,roomb,roomc,roomwinner,RoomPred,SpacePred
0,106,106,101,103,rooma,rooma,106
1,106,106,101,103,rooma,rooma,106
2,103,106,101,103,roomc,roomc,103
3,102,101,106,103,none,none,none
5,105,106,101,103,none,none,none
6,103,106,101,103,roomc,roomc,103
7,101,106,101,103,roomb,roomb,101
8,112,106,112,101,roomb,none,none
9,103,103,101,106,rooma,rooma,103
10,104,104,103,106,rooma,rooma,104


In [300]:
cleandata28[['roomwinner','RoomPred','SpacePred']].groupby(['roomwinner','RoomPred']).agg('count')
#over half are 'none' which is obviously wrong, explains why space pred score is so low
#and why the room pred score is so much higher
#this is a failure of the munged metric approach
#at least the prediction of a proxy variable actually works; it is something that did not resolve well in R

SpacePred
roomwinner RoomPred           
none       none          10840
           rooma           237
           roomb           119
           roomc            76
rooma      none            305
           rooma          3747
           roomb            72
           roomc            32
roomb      none            201
           rooma           108
           roomb          2293
           roomc            17
roomc      none            225
           rooma            36
           roomb            54
           roomc          1499

In [301]:
#memory at a bit under a gig
#finally for fourth, relative position
#this one will be just like building or floor, except only two choices

In [302]:
#first, the roomdata table, with as_index = False
relposdata = WAPdata.iloc[:,0:471].drop(['LONGITUDE','LATITUDE'], axis=1).groupby(['BUILDINGID','FLOOR','SPACEID','RELATIVEPOSITION'], as_index = False).sum(axis=0).copy()
print(relposdata)

     BUILDINGID  FLOOR  SPACEID  RELATIVEPOSITION  WAP001  WAP002  WAP005  \
0             0      0      102                 2       0       0       0   
1             0      0      106                 2       0       0       0   
2             0      0      107                 2       0       0       0   
3             0      0      110                 2       0       0       0   
4             0      0      111                 2       0       0       0   
5             0      0      112                 2       0       0       0   
6             0      0      113                 2       0       0       0   
7             0      0      114                 2       0       0       0   
8             0      0      115                 2       0       0       0   
9             0      0      116                 2       0       0       0   
10            0      0      117                 2       0       0       0   
11            0      0      118                 2       0       0       0   

In [303]:
relposdata.loc[(relposdata['BUILDINGID']==1) & (relposdata['FLOOR']==0) & (relposdata['SPACEID']==107)]

,BUILDINGID,FLOOR,SPACEID,RELATIVEPOSITION,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,...,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519
269,1,0,107,2,0,0,0,0,0,20,...,0,0,0,0,0,0,0,0,0,0


In [304]:
relposdata.loc[(relposdata['BUILDINGID']==1) & (relposdata['FLOOR']==0) & (relposdata['SPACEID']==107)].shape[0]

1

In [305]:
relpostemp = relposdata.loc[(relposdata['BUILDINGID']==1) & (relposdata['FLOOR']==0) & (relposdata['SPACEID']==107)]  
tempvars7 = applWAPs(cleandata28.iloc[0,0:465])
tempvars8 = tempvars7.append(pd.Index(list({'RELATIVEPOSITION'})))
relpostemp[tempvars8]

,WAP090,WAP091,WAP103,WAP104,WAP105,WAP106,WAP125,WAP126,WAP166,WAP167,WAP168,WAP169,WAP171,WAP172,WAP173,WAP191,WAP192,RELATIVEPOSITION
269,0,0,8,17,3,5,20,20,0,0,0,0,0,0,0,0,0,2


In [306]:
relpostemp[tempvars8].groupby(['RELATIVEPOSITION']).agg('sum',axis=1).sum(axis=1)

RELATIVEPOSITION
2    73
dtype: int64

In [319]:
#function to return the relposdata
#this only works if there are two possible relative positions; fails if there is only 1

def relpossum2 (observation):
    bldg = observation['BldgPred']
    flr = observation['FlrPred']
    space = observation['SpacePred']
    relpostemp = relposdata.loc[(relposdata['BUILDINGID']==bldg) & (relposdata['FLOOR']==flr) & (relposdata['SPACEID']==space)]
    tempvars9 = applWAPs(observation.iloc[0:465])
    tempvars10 = tempvars9.append(pd.Index(list({'RELATIVEPOSITION'})))
    rpsums = relpostemp[tempvars10].groupby(['RELATIVEPOSITION']).agg('sum',axis=1).sum(axis=1)
    rpsums2 = pd.DataFrame(data = rpsums)
    return rpsums2.iloc[0].max(), rpsums2.iloc[1].max(), rpsums2.iloc[0].max()/rpsums2.iloc[1].max()

In [331]:
def relpossum (observation):
    bldg = observation['BldgPred']
    flr = observation['FlrPred']
    space = observation['SpacePred']
    if space == "none":
        return -1, 2, -1
    else: 
        relpostemp = relposdata.loc[(relposdata['BUILDINGID']==bldg) & (relposdata['FLOOR']==flr) & (relposdata['SPACEID']==space)]
    if relpostemp.shape[0] == 1:
        return -1, relpostemp['RELATIVEPOSITION'].max(), -1
    else:
        return relpossum2(observation)

In [332]:
#test
relpossum(cleandata28.iloc[0,0:509])

(127, 579, 0.21934369602763384)

In [333]:
#test
relpossum(cleandata28.iloc[3,0:509])

(-1, 2, -1)

In [334]:
cleandata28[['BldgPred','FlrPred','SpacePred','RELATIVEPOSITION']].head(13)
#none throws off, modified above to return a default relpos (see below)

,BldgPred,FlrPred,SpacePred,RELATIVEPOSITION
0,1,2,106,2
1,1,2,106,2
2,1,2,103,2
3,1,2,none,2
5,1,2,none,2
6,1,2,103,2
7,1,2,101,2
8,1,2,none,2
9,1,2,103,1
10,1,2,104,1


In [335]:
cleandata28[['SpacePred','RELATIVEPOSITION','WAP001']].groupby(['SpacePred','RELATIVEPOSITION']).agg('count')
#I will default to '2' for nones

WAP001
SpacePred RELATIVEPOSITION        
1         1                      3
          2                     49
2         1                     23
          2                     60
3         1                      8
          2                     35
4         1                     15
          2                     73
5         1                      9
          2                     17
6         1                     12
          2                     41
7         1                      8
          2                     16
8         1                      5
          2                     33
9         1                      8
          2                     20
10        1                     14
          2                     53
13        2                     65
101       1                     61
          2                    123
102       1                     58
          2                    155
103       1                     63
          2                    122
104       1                     89
          2                    155
105       1                     89
...                            ...
225       2                     19
226       2                      2
227       2                     17
228       2                     18
229       2                     52
230       1                      6
          2                     71
231       2                     67
233       2                     14
234       2                     23
235       2                     38
236       1                      5
          2                     57
237       1                      2
          2                      2
238       1                      7
          2                      9
241       2                     55
242       1                      9
          2                     23
243       1                     10
          2                     20
244       1                     32
          2                     14
247       1                     60
          2                     30
248       1                     75
          2                     17
none      1                   1296
          2                  10275

[170 rows x 1 columns]

In [336]:
#apply method to get all
cleandata28.iloc[0:13,0:509].apply(relpossum,axis=1)

0     (127, 579, 0.21934369602763384)
1     (130, 525, 0.24761904761904763)
2     (177, 360, 0.49166666666666664)
3                         (-1, 2, -1)
5                         (-1, 2, -1)
6     (177, 406, 0.43596059113300495)
7      (119, 496, 0.2399193548387097)
8                         (-1, 2, -1)
9      (172, 230, 0.7478260869565218)
10     (229, 210, 1.0904761904761904)
11    (124, 505, 0.24554455445544554)
12                        (-1, 2, -1)
13                        (-1, 2, -1)
dtype: object

In [337]:
#whole enchilada
cleandata29 = pd.concat((cleandata28, cleandata28.iloc[:,0:509].apply(relpossum,axis=1).rename('temp')),axis=1).copy()
print(cleandata29.head(6))

#clear variable
#cleandata28 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012               ...                 roomc  roomcsum  roomtotal  \
0     100               ...                   103       562       4244   
1     100               ...                   103       444       3697   
2     100               ...                   103       537       3745   
3     100               ...                   103       509       3828   
5     100               ...                   103       496       3342   
6     100       

In [338]:
cleandata30=pd.concat((cleandata29, cleandata29.loc[:,'temp'].apply(pd.Series,index=["relpos1","relpos2","relposratio"])), axis=1).copy()
print(cleandata30.head(6))

#clear variable
cleandata29 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012     ...       roomaratio  roombratio  roomcratio  roomwinner  \
0     100     ...         0.166352    0.161876    0.132422       rooma   
1     100     ...         0.177171    0.160130    0.120097       rooma   
2     100     ...         0.159413    0.157009    0.143391       roomc   
3     100     ...         0.152560    0.146290    0.132968        none   
5     100     ...         0.177738    0.169958    0.148414        none   
6     100     ..

In [339]:
cleandata30.drop('temp', axis=1, inplace=True)
cleandata30.head(6)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,roomtotal,roomaratio,roombratio,roomcratio,roomwinner,RoomPred,SpacePred,relpos1,relpos2,relposratio
0,100,100,100,100,100,100,100,100,100,100,...,4244,0.166352,0.161876,0.132422,rooma,rooma,106,127.0,579.0,0.219344
1,100,100,100,100,100,100,100,100,100,100,...,3697,0.177171,0.160130,0.120097,rooma,rooma,106,130.0,525.0,0.247619
2,100,100,100,100,100,-97,100,100,100,100,...,3745,0.159413,0.157009,0.143391,roomc,roomc,103,177.0,360.0,0.491667
3,100,100,100,100,100,100,100,100,100,100,...,3828,0.152560,0.146290,0.132968,none,none,none,-1.0,2.0,-1.000000
5,100,100,100,100,100,100,100,100,100,100,...,3342,0.177738,0.169958,0.148414,none,none,none,-1.0,2.0,-1.000000
6,100,100,100,100,100,100,100,100,100,100,...,4483,0.155476,0.153469,0.130047,roomc,roomc,103,177.0,406.0,0.435961


In [340]:
#now for room model stuff

#dependent variable
depVarrp = cleandata30.loc[:,'RELATIVEPOSITION'].copy()

#features
featuresrp = cleandata30.iloc[:,509:512].copy()
print(featuresrp.head(6))

#make train and test sets
rpxtrain, rpxtest, rpytrain, rpytest = train_test_split(featuresrp, depVarrp, random_state = 123) #added random state so reproduceable
print(rpxtrain.shape)
print(rpxtest.shape)

   relpos1  relpos2  relposratio
0    127.0    579.0     0.219344
1    130.0    525.0     0.247619
2    177.0    360.0     0.491667
3     -1.0      2.0    -1.000000
5     -1.0      2.0    -1.000000
6    177.0    406.0     0.435961
(14895, 3)
(4966, 3)


In [280]:
#models below
#results will be commented, cells removed from active processing for future reruns

#random forest
rf_clf = make_pipeline(RandomForestClassifier(random_state = 123))
param_grid = {'randomforestclassifier__n_estimators': np.arange(1, 26, 5),
             'randomforestclassifier__criterion': ["gini", "entropy"]}
rprf_cv = GridSearchCV(rf_clf, param_grid, cv = 10)
%timeit rprf_cv.fit(rpxtrain, rpytrain)
print("Tuned rf parameters: {}".format(rprf_cv.best_params_))
print("Best score is {}".format(rprf_cv.best_score_))
#print(sorted(rprf_clf.get_params().keys()))

#results
#7.31 s ± 20.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned rf parameters: {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__n_estimators': 21}
#Best score is 0.8867405169519973

#gradient boosting
#nerfed this last one so it would go faster
gb_clf = make_pipeline(GradientBoostingClassifier(random_state = 123))
param_grid_gb = { 
        'gradientboostingclassifier__n_estimators': [3], 
        'gradientboostingclassifier__learning_rate': [.05, .1, .5, 1.0],
        'gradientboostingclassifier__max_depth': [3]}
rpgb_cv = GridSearchCV(gb_clf, param_grid_gb, cv = 10)
%timeit rpgb_cv.fit(rpxtrain, rpytrain)
print("Tuned gb parameters: {}".format(rpgb_cv.best_params_))
print("Best score is {}".format(rpgb_cv.best_score_))
#print(sorted(rpgb_clf.get_params().keys()))

#results
#691 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned gb parameters: {'gradientboostingclassifier__learning_rate': 1.0, 'gradientboostingclassifier__max_depth': 3, 'gradientboostingclassifier__n_estimators': 3}
#Best score is 0.8757972473984559

#knn
#nerfed for speed
knn_clf = make_pipeline(KNeighborsClassifier())
param_grid_knn = { 
        'kneighborsclassifier__n_neighbors': [3, 5] 
                }
rpknn_cv = GridSearchCV(knn_clf, param_grid_knn, cv = 10)
%timeit rpknn_cv.fit(rpxtrain, rpytrain)
print("Tuned knn parameters: {}".format(rpknn_cv.best_params_))
print("Best score is {}".format(rpknn_cv.best_score_))
#print(sorted(rpknn_clf.get_params().keys()))

#results
#29.6 s ± 325 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned knn parameters: {'kneighborsclassifier__n_neighbors': 5}
#Best score is 0.8823094998321585

#decision tree, allows for visualization if can figure out graphviz
dt_clf = make_pipeline(DecisionTreeClassifier(random_state = 123))
param_grid = {'decisiontreeclassifier__criterion': ["gini", "entropy"]}
rpdt_cv = GridSearchCV(dt_clf, param_grid, cv = 10)
%timeit rpdt_cv.fit(rpxtrain, rpytrain)
print("Tuned dt parameters: {}".format(rpdt_cv.best_params_))
print("Best score is {}".format(rpdt_cv.best_score_))
#print(sorted(rpdt_clf.get_params().keys()))

#results
#395 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#Tuned dt parameters: {'decisiontreeclassifier__criterion': 'entropy'}
#Best score is 0.8828465928163813

In [281]:
#pause, to hopefully let decision tree unstar while svc is still going

#svc
svc_clf = make_pipeline(SVC(random_state = 123))
param_grid_svc = { 
        'svc__kernel': ['linear','rbf'],
        'svc__C': [1, 10],
                    }
rpsvc_cv = GridSearchCV(svc_clf, param_grid_svc, cv = 10)
%timeit rpsvc_cv.fit(rpxtrain, rpytrain)
print("Tuned svc parameters: {}".format(rpsvc_cv.best_params_))
print("Best score is {}".format(rpsvc_cv.best_score_))
#print(sorted(rpsvc_clf.get_params().keys()))

#results
#just skipping it

In [347]:
#now models using optimal settings
rpmodelRF = RandomForestClassifier(n_estimators=21, criterion = 'gini', random_state=123)
rpmodelGB = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=3, random_state=123)
rpmodelKNN = KNeighborsClassifier(n_neighbors=5)
rpmodelDT = DecisionTreeClassifier(criterion = 'entropy', random_state=123)
#rpmodelSVC = SVC(C=1, kernel='linear', random_state=123)

In [348]:
#the fit
rpmodelRF.fit(rpxtrain, rpytrain)
rpmodelGB.fit(rpxtrain, rpytrain)
rpmodelKNN.fit(rpxtrain, rpytrain)
rpmodelDT.fit(rpxtrain, rpytrain)
#rpmodelSVC.fit(rpxtrain, rpytrain)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [349]:
#now predict
rppredictionsRF = rpmodelRF.predict(rpxtest)
rppredictionsGB = rpmodelGB.predict(rpxtest)
rppredictionsKNN = rpmodelKNN.predict(rpxtest)
rppredictionsDT = rpmodelDT.predict(rpxtest)
#rppredictionsSVC = rpmodelSVC.predict(rpxtest)

In [350]:
#score
print(accuracy_score(rpytest, rppredictionsRF))
print(accuracy_score(rpytest, rppredictionsGB))
print(accuracy_score(rpytest, rppredictionsKNN))
print(accuracy_score(rpytest, rppredictionsDT))
#print(accuracy_score(rpytest, rppredictionsSVC))

#results
#RF 0.8741441804269029
#GB 0.8348771647200967
#KN 0.8683044703987113
#DT 0.8683044703987113
#SV 

#compare to train results
#RF 0.8867405169519973
#GB 0.8757972473984559
#KN 0.8823094998321585
#DT 0.8828465928163813
#SV 

#rf is the most accurate and consistent (less chance overfit)
#I believe most of the error here is due to the Space = none being put to 2 by default; 
#a good assumption but off 11% of the nones, which are about half of the data

0.8741441804269029
0.8348771647200967
0.8683044703987113
0.8683044703987113


In [351]:
rppredictionsRF

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [352]:
RelPosPred = rpmodelRF.predict(featuresrp)

In [353]:
RelPosPred.shape

(19861,)

In [354]:
RelPosPredpd = pd.DataFrame({'RelPosPred':RelPosPred}, index = cleandata30.index)
RelPosPredpd.info

<bound method DataFrame.info of        RelPosPred
0               2
1               2
2               2
3               2
5               2
6               2
7               2
8               2
9               1
10              1
11              2
12              2
13              2
14              2
15              2
16              1
17              2
18              2
19              2
20              2
21              2
22              2
23              2
24              2
25              2
26              2
27              1
28              1
29              2
30              2
...           ...
19907           2
19908           1
19909           2
19910           2
19911           2
19912           2
19913           2
19914           2
19915           2
19916           2
19917           2
19918           2
19919           2
19920           2
19921           2
19922           2
19923           2
19924           2
19925           1
19926           2
19927           2
19928         

In [355]:
RelPosPredpd.shape

(19861, 1)

In [356]:
cleandata30.shape

(19861, 512)

In [357]:
#Make room prediction
cleandata31=pd.concat((cleandata30, RelPosPredpd.loc[:,'RelPosPred']),axis=1).copy()

print(cleandata30.head(6))
print(cleandata30.shape)
print(cleandata31.head(6))
print(cleandata31.shape)

#clear variable
cleandata30 = None

   WAP001  WAP002  WAP005  WAP006  WAP007  WAP008  WAP009  WAP010  WAP011  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     -97     100     100     100   
3     100     100     100     100     100     100     100     100     100   
5     100     100     100     100     100     100     100     100     100   
6     100     100     100     100     100     100     100     100     100   

   WAP012     ...       roomtotal  roomaratio  roombratio  roomcratio  \
0     100     ...            4244    0.166352    0.161876    0.132422   
1     100     ...            3697    0.177171    0.160130    0.120097   
2     100     ...            3745    0.159413    0.157009    0.143391   
3     100     ...            3828    0.152560    0.146290    0.132968   
5     100     ...            3342    0.177738    0.169958    0.148414   
6     100     ...     

In [360]:
#how many relative position predictions actually differ from the correct

relpospredictmatch = pd.DataFrame({'Match': cleandata31.loc[:,'RELATIVEPOSITION'] == cleandata31.loc[:,'RelPosPred']})
print(relpospredictmatch.shape)
relpospredictmatch.groupby('Match').size()

#results
#Match
#False      1616     ~ 8.14%
#True      18245
#A bit lower than the training, there is some overfit

(19861, 1)


Match
False     1616
True     18245
dtype: int64

In [362]:
type(cleandata31)

pandas.core.frame.DataFrame

In [367]:
#because of the troubles with space ID, our overall accuracy will be similarly abysmal

cleandata31.loc[(cleandata31['BUILDINGID']==cleandata31['BldgPred']) & (cleandata31['FLOOR']==cleandata31['FlrPred']) & (cleandata31['SPACEID']==cleandata31['SpacePred'])& (cleandata31['RELATIVEPOSITION']==cleandata31['RelPosPred'])].agg('count').min()

#results
#Match
#False     12505      ~ 62.96%
#True       7356      ~ 37.04%

#terrible accuracy, but expected

7356